# Phase 1

In [2]:
import os
import pandas as pd

## wissel-ariregister

In [3]:
df_aziende = pd.read_csv("./datasets/wissel-ariregister.rik.ee/wissel-aziende-ariregister.rik.ee.csv")
df_activity = pd.read_csv("./datasets/wissel-ariregister.rik.ee/wissel-activity-ariregister.rik.ee.csv")
df_partners = pd.read_csv("./datasets/wissel-ariregister.rik.ee/wissel-partners-ariregister.rik.ee.csv")
df_rappresentanti = pd.read_csv("./datasets/wissel-ariregister.rik.ee/wissel-rappresentanti-ariregister.rik.ee.csv")

In [4]:
df_aziende.rename(columns={'ID':'ID azienda'}, inplace=True)
toDrop = ["URL", "Code", "Legal form", "Status", "Registration Date", "Deletion Date"]
df_aziende = df_aziende.drop(toDrop, axis=1)
df_aziende.head()

,ID azienda,Name,Capital,Address
0,1,10DspaceOÃœ(14318875),NaN,NaN
1,2,21SparrowsOÃœ(16270291),Capitalis10Â 000Â â‚¬,Harju maakond Tallinn ...
2,3,3DSTOÃœ(12454634),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...
3,4,8BitsSpaceOÃœ(14867109),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...
4,5,A1EhituspartnerOÃœ(16451101),Capitalis2Â 500Â â‚¬,PÃµlva maakond PÃµlva ...


In [5]:
toDrop = ["EMTAK Code", "NACE Code", "Source"]
df_activity = df_activity.drop(toDrop, axis=1)
df_activity.head()

,ID azienda,Area of Activity
0,2,Other activities auxiliary to financial servic...
1,3,Specialised design activities
2,4,Computer programming activities
3,5,Construction of residential and non-residentia...
4,6,Renting and operational leasing of other machi...


In [6]:
df_partners.head()

,ID azienda,Name,Code,Participation,Contribution
0,2,\n Jani...,38807314918\n,70.00%,\n 7000.00 ...
1,2,\n Kair...,48812154914\n,30.00%,\n 3000.00 ...
2,3,\n Siim...,38807042731\n,100.00%,\n 2500.00 ...
3,4,\n Domi...,39207230150\n,100.00%,\n 2500.00 ...
4,5,\n Arti...,38205186529\n,50.00%,\n 1250.00 ...


In [7]:
df_rappresentanti.head()

,ID azienda,Name,Code,Role,Start Date
0,2,\n Janis Jegi\n...,\n 38807314...,\n Management b...,14.07.2021
1,3,\n Siim Tiigim\...,\n 38807042...,\n Management b...,15.04.2013
2,4,\n Dominik Gron...,\n 39207230...,\n Management b...,11.12.2019
3,5,\n Rainis Luik\...,\n 37508026...,\n Management b...,28.02.2022
4,6,\n Kaido Kaabel...,\n 36706040...,\n Management b...,28.11.1996


In [8]:
df3 = pd.merge(df_aziende, df_activity, how='left', on='ID azienda')

In [9]:
df3.head()

,ID azienda,Name,Capital,Address,Area of Activity
0,1,10DspaceOÃœ(14318875),NaN,NaN,NaN
1,2,21SparrowsOÃœ(16270291),Capitalis10Â 000Â â‚¬,Harju maakond Tallinn ...,Other activities auxiliary to financial servic...
2,3,3DSTOÃœ(12454634),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...,Specialised design activities
3,4,8BitsSpaceOÃœ(14867109),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...,Computer programming activities
4,5,A1EhituspartnerOÃœ(16451101),Capitalis2Â 500Â â‚¬,PÃµlva maakond PÃµlva ...,Construction of residential and non-residentia...


In [10]:
df3.to_csv("./datasets/wissel-ariregister.rik.ee.csv")

## wissel-companiesmarketcap

In [11]:
df_companies_aziende = pd.read_csv("./datasets/wissel-companiesmarketcap.com/wissel-aziende-companiesmarketcap.com.csv")
df_companies_categorie = pd.read_csv("./datasets/wissel-companiesmarketcap.com/wissel-categorie-companiesmarketcap.com.csv")

In [12]:
df_companies_aziende.head()

,Unnamed: 0,URL,ID,Name,Company code,Marketcap,Share price,Earnings,Revenue,Shares,Employees
0,0,https://companiesmarketcap.com/vertex-pharmace...,1.0,Vertex Pharmaceuticals,VRTX,$77.24 B,$300.94,$4.08 B,$8.70 B,256.500.000,3.9
1,1,https://companiesmarketcap.com/verano-holdings...,2.0,Verano Holdings,VRNOF,$0.97 B,$3.00,-$62.2 M,$0.81 B,NaN,3.888
2,2,https://companiesmarketcap.com/azz/marketcap/,3.0,AZZ,AZZ,$1.09 B,$43.85,$0.12 B,$1.17 B,NaN,3.885
3,3,https://companiesmarketcap.com/fnb-corp/market...,4.0,F.N.B. Corporation,FNB,$4.83 B,$13.41,$0.50 B,$1.33 B,NaN,3.884
4,4,https://companiesmarketcap.com/quanex-building...,5.0,Quanex Building Products,NX,$0.82 B,$24.96,$0.10 B,$1.22 B,NaN,3.86


In [13]:
toDrop = ["Unnamed: 0", "URL"]
df_companies_aziende = df_companies_aziende.drop(toDrop, axis=1)
df_companies_aziende.head()

,ID,Name,Company code,Marketcap,Share price,Earnings,Revenue,Shares,Employees
0,1.0,Vertex Pharmaceuticals,VRTX,$77.24 B,$300.94,$4.08 B,$8.70 B,256.500.000,3.9
1,2.0,Verano Holdings,VRNOF,$0.97 B,$3.00,-$62.2 M,$0.81 B,NaN,3.888
2,3.0,AZZ,AZZ,$1.09 B,$43.85,$0.12 B,$1.17 B,NaN,3.885
3,4.0,F.N.B. Corporation,FNB,$4.83 B,$13.41,$0.50 B,$1.33 B,NaN,3.884
4,5.0,Quanex Building Products,NX,$0.82 B,$24.96,$0.10 B,$1.22 B,NaN,3.86


In [14]:
df_companies_categorie.head()

,Company ID,Category,Hash
0,1,Retail,2932210656109287952
1,1,Dowjones,-3512652547975320346
2,1,SupermarketChains,-4069439822336842578
3,2,ECommerce,5458971525214020641
4,2,Internet,-848597773130649638


##### Company ID e ID non corrispondono

In [15]:
df_companies_aziende.to_csv("./datasets/wissel-companiesmarketcap.com.csv")

## FR_sole24ore.csv

In [16]:
fr_sole24 = pd.read_csv("./datasets/FR_sole24ore.csv")
if fr_sole24.columns[len(fr_sole24.columns)-1] == "name":
    fr_sole24 = fr_sole24.drop(fr_sole24.columns[0], axis=1)
else:
    fr_sole24 = pd.read_csv("./datasets/FR_sole24ore.csv", header=None)

In [17]:
fr_sole24.columns = ["address", "country", "sector", "employees", "name"]
fr_sole24.head()

,address,country,sector,employees,name
0,Via Maurizio Gonzaga 5,NaN,E-commerce,7.337,Winelivery S.r.l.
1,Via Damiano Chiesa 53,Bitritto,Servizi generali,8.375,Pegaso Security S.r.l.
2,Piazza IV Novembre 4,Milano,Vendita al dettaglio & all'ingrosso,13.889,Euro Top Brand S.r.l.s.
3,Piazza Manifattura 1,Rovereto,Energia,3.585,Infinityhub S.p.A. Benefit
4,Via Solferino 36,Milano,Servizi finanziari,11.302,Credimi S.p.a.


In [18]:
fr_sole24.to_csv("./datasets/FR_sole24ore.csv")

## Files

In [19]:
source = 'datasets/'
files_list = os.listdir(source)
files_list.sort()
files_list = [source + x for x in files_list if os.path.isfile(source+x)]

In [20]:
# print(files_list)
final_files = []
len(files_list)

49

In [21]:
file_ext = {}
columns_file = {}
files_columns = {}
for i in files_list:
    #print("File name: " + i)
    #print(i[len(i)-3:])
    if i[len(i)-3:] == "csv":
        if "csv" not in file_ext:
            file_ext["csv"] = 0
        file_ext["csv"] += 1

        current_file = pd.read_csv(i, encoding="utf-8", encoding_errors="ignore")
        # print(file_csv.columns)


    elif i[len(i)-4:] == "json":
        if "json" not in file_ext:
            file_ext["json"] = 0
        file_ext["json"] += 1

        current_file = pd.read_json(i)
        # print(file_json.head())
        
    elif i[len(i)-5:] == "jsonl":
        if "jsonl" not in file_ext:
            file_ext["jsonl"] = 0
        file_ext["jsonl"] += 1

        current_file = pd.read_json(i, lines=True)
        # print(file_jsonl.head())
        
    elif i[len(i)-3:] == "xls":
        if "xls" not in file_ext:
            file_ext["xls"] = 0
        file_ext["xls"] += 1

        current_file = pd.read_excel(i)
        # print(file_xls.head())
        
    elif i[len(i)-4:] == "xlsx":
        if "xlsx" not in file_ext:
            file_ext["xlsx"] = 0
        file_ext["xlsx"] += 1
        
        # Genera warning
        current_file = pd.read_excel(i)
        # print(file_xlsx.head())
        
    else:
        print("file error: " + i)

    if i not in files_columns:
        files_columns[i] = current_file.columns


    for c in current_file.columns:
        if c.lower() not in columns_file:
            columns_file[c.lower()] = 0
        columns_file[c.lower()] += 1

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [22]:
file_ext

{'csv': 25, 'json': 16, 'xls': 3, 'xlsx': 1, 'jsonl': 4}

In [23]:
files_columns

{'datasets/DDD-cbinsight.com.csv': Index(['Unnamed: 0', 'name', 'valuation', 'dateJoined', 'country', 'city',
        'industry', 'investors', 'founded', 'stage', 'totalRaised'],
       dtype='object'),
 'datasets/DDD-companiesmarketcap.com.csv': Index(['Unnamed: 0', 'name', 'rank', 'marketcap', 'country', 'share_price',
        'change1d', 'change1y', 'categories'],
       dtype='object'),
 'datasets/DDD-ft.com.csv': Index(['Unnamed: 0', 'name', 'country', 'sector', 'absolute_growth_rate_pct',
        'compound_annual_growth_rate_cagr_pct', 'revenue_2020_euro',
        'revenue_2017_euro', 'number_of_employees_2020',
        'number_of_employees_2017', 'founding_year'],
       dtype='object'),
 'datasets/DDD-teamblind.com.csv': Index(['Unnamed: 0', 'name', 'website', 'locations', 'size', 'industry',
        'founded'],
       dtype='object'),
 'datasets/FR_campaignindia.csv': Index(['RANK', 'BRAND NAME', 'CATEGORY', 'MAIN MARKET', '2010 RANK'], dtype='object'),
 'datasets/FR_company_s

## Check columns

In [24]:
import json
# with open('columns_to_remove.json', 'w', encoding='utf8') as f:
#     json.dump(columns_to_remove, f, ensure_ascii=False)

# with open('columns_to_rename.json', 'w', encoding='utf8') as f:
#     json.dump(columns_to_rename, f, ensure_ascii=False)

In [25]:
def columns_to_remove_not_remove(columns):
    with open('columns_to_remove.json', 'r', encoding='utf8') as f:
        columns_to_remove = json.load(f)

    r = []
    l = []
    print("Columns to remove:")
    for c in columns:
        if c.lower() in columns_to_remove:
            print(c)
            r.append(c)
        else:
            l.append(c)
    return (r, l)

In [26]:
def rename_columns(columns):
    with open('columns_to_rename.json', 'r', encoding='utf8') as f:
        columns_to_rename = json.load(f)
    
    l = []
    print("Columns to rename:")
    for c in columns:
        error=True
        for name_column, list_strings in columns_to_rename.items():
            if c.lower() == name_column:
                l.append(name_column)
                error=False
            elif c.lower() in list_strings:
                print(c + " -> " + name_column)
                l.append(name_column)
                error=False    
        
        if error: 
            l.append(c.lower())
            print("Column error: " + c)
    return l

In [27]:
# columns_final_file = ["name", "country", "sector", "marketcap", "revenue", "employees", "ceo", "founded", "links"]

In [28]:
cleaned_files = []

## DDD

### DDD-cbinsight.com.csv

In [29]:
files_list[0]

'datasets/DDD-cbinsight.com.csv'

In [30]:
ddd_cbinsight = pd.read_csv(files_list[0])
ddd_cbinsight.head()

,Unnamed: 0,name,valuation,dateJoined,country,city,industry,investors,founded,stage,totalRaised
0,0,Lacework,$8.3B,1/7/2021,United States,San Jose,Cybersecurity,"Sutter Hill Ventures, Liberty Global Ventures,...",2015,Series E,$1.907B
1,1,Tipalti,$8.3B,10/6/2020,United States,San Mateo,Fintech,"01 Advisors, Zeev Ventures, Group 11",2010,Series F,$556M
2,2,Tempus,$8.1B,3/21/2018,United States,Chicago,Health,"New Enterprise Associates, T. Rowe Associates,...",2015,Unattributed - II,$1.345B
3,3,Anduril,$8.48B,9/11/2019,United States,Irvine,Artificial intelligence,"Andreessen Horowitz, Founders Fund, Revolution...",2017,Series E,$2.309B
4,4,Bolt,$8.4B,5/29/2018,Estonia,Tallinn,Auto & transportation,"Didi Chuxing, Diamler, TMT Investments",2013,Series F,$1.971B


In [31]:
to_drop, _ = columns_to_remove_not_remove(ddd_cbinsight)

Columns to remove:
Unnamed: 0
valuation
dateJoined
city
investors
stage
totalRaised


In [32]:
ddd_cbinsight = ddd_cbinsight.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_cbinsight.columns)
ddd_cbinsight.columns = to_rename
ddd_cbinsight.head()

Columns to rename:


,name,country,industry,founded
0,Lacework,United States,Cybersecurity,2015
1,Tipalti,United States,Fintech,2010
2,Tempus,United States,Health,2015
3,Anduril,United States,Artificial intelligence,2017
4,Bolt,Estonia,Auto & transportation,2013


In [33]:
ddd_cbinsight.head()

,name,country,industry,founded
0,Lacework,United States,Cybersecurity,2015
1,Tipalti,United States,Fintech,2010
2,Tempus,United States,Health,2015
3,Anduril,United States,Artificial intelligence,2017
4,Bolt,Estonia,Auto & transportation,2013


In [43]:
import numpy as np

In [52]:
ddd_cbinsight = ddd_cbinsight.replace('None', np.nan)
ddd_cbinsight.founded = ddd_cbinsight.founded.astype(object).astype('Int64')

In [33]:
cleaned_files.append(ddd_cbinsight)

### DDD-companiesmarketcap.com.csv

In [54]:
files_list[1]

'datasets/DDD-companiesmarketcap.com.csv'

In [55]:
ddd_companies = pd.read_csv(files_list[1])
ddd_companies.head()

,Unnamed: 0,name,rank,marketcap,country,share_price,change1d,change1y,categories
0,0,Apple,1,$2.194 T,usa,$137.98,1.50%,-18.74%,tech software dow-jones tech-hardware electron...
1,1,Unilever,99,$129.09 B,united-kingdom,$50.98,-0.21%,-6.09%,food consumer-goods
2,2,AIA,97,$130.12 B,hong-kong,$11.11,1.34%,3.93%,insurance
3,3,Lowe's Companies,100,$128.96 B,usa,$213.26,1.79%,-12.38%,retail
4,4,RWE,601,$30.02 B,germany,$44.41,-1.16%,6.40%,electricity dax energy


In [57]:
to_drop, _ = columns_to_remove_not_remove(ddd_companies)

Columns to remove:
Unnamed: 0
rank
share_price
change1d
change1y


In [58]:
ddd_companies = ddd_companies.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_companies.columns)
ddd_companies.columns = to_rename
ddd_companies.head()

Columns to rename:
categories -> industry


,name,marketcap,country,industry
0,Apple,$2.194 T,usa,tech software dow-jones tech-hardware electron...
1,Unilever,$129.09 B,united-kingdom,food consumer-goods
2,AIA,$130.12 B,hong-kong,insurance
3,Lowe's Companies,$128.96 B,usa,retail
4,RWE,$30.02 B,germany,electricity dax energy


In [67]:
ddd_companies.isnull().sum()

name            0
marketcap      17
country       196
industry     1171
dtype: int64

In [66]:
ddd_companies = ddd_companies.replace('None', np.nan)

In [63]:
if "T" in :
    print("eeeee")

eeeee


In [70]:
d = "$2.194 T"

In [73]:
if d[len(d)-1] != "T" and d[len(d)-1] != "B":
    print("dio cane")

In [79]:
x = 0
for d in ddd_companies.marketcap:
    try: 
        if d[len(d)-1] != "T" and d[len(d)-1] != "B" and d[len(d)-1] != "M":
            print(d)
    except:
        print(d)


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [38]:
cleaned_files.append(ddd_companies)

### DDD-ft.com.csv

In [80]:
files_list[2]

'datasets/DDD-ft.com.csv'

In [114]:
ddd_ft = pd.read_csv(files_list[2])
ddd_ft.head()

,Unnamed: 0,name,country,sector,absolute_growth_rate_pct,compound_annual_growth_rate_cagr_pct,revenue_2020_euro,revenue_2017_euro,number_of_employees_2020,number_of_employees_2017,founding_year
0,0,Swappie,Finland,Technology,"19,152.82",477.43,"97,611,814","507,000",218,1,2016
1,1,Kilo Health,Lithuania,Health,"16,541.73",450.05,"57,318,766","344,428",177,10,2013
2,2,OCI,UK,Financial Services,"13,133.13",409.59,"568,322,073","4,325,512",32,4,2012
3,3,OnlyFans,UK,Technology,"11,928.05",393.63,"316,732,986","2,652,185",800,4,2016
4,4,Enpal,Germany,Energy,"11,441.27",386.88,"56,109,613","486,165",365,9,2017


In [115]:
to_drop, _ = columns_to_remove_not_remove(ddd_ft)

Columns to remove:
Unnamed: 0
absolute_growth_rate_pct
compound_annual_growth_rate_cagr_pct
revenue_2017_euro
number_of_employees_2017


In [116]:
ddd_ft = ddd_ft.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_ft.columns)
ddd_ft.columns = to_rename
ddd_ft.head()

Columns to rename:
Column error: sector
revenue_2020_euro -> revenue
number_of_employees_2020 -> employees
founding_year -> founded


,name,country,sector,revenue,employees,founded
0,Swappie,Finland,Technology,"97,611,814",218,2016
1,Kilo Health,Lithuania,Health,"57,318,766",177,2013
2,OCI,UK,Financial Services,"568,322,073",32,2012
3,OnlyFans,UK,Technology,"316,732,986",800,2016
4,Enpal,Germany,Energy,"56,109,613",365,2017


In [117]:
ddd_ft = ddd_ft.replace('None', np.nan)
ddd_ft.revenue = ddd_ft.revenue.apply(lambda x : x.replace(',',''))

ddd_ft.revenue = ddd_ft.revenue.astype(object).astype('Int64')
ddd_ft.employees = ddd_ft.employees.astype(object).astype('Int64')
ddd_ft.founded = ddd_ft.founded.astype(object).astype('Int64')
ddd_ft.head()

,name,country,sector,revenue,employees,founded
0,Swappie,Finland,Technology,97611814,218,2016
1,Kilo Health,Lithuania,Health,57318766,177,2013
2,OCI,UK,Financial Services,568322073,32,2012
3,OnlyFans,UK,Technology,316732986,800,2016
4,Enpal,Germany,Energy,56109613,365,2017


In [43]:
cleaned_files.append(ddd_ft)

### DDD-teamblind.com.csv

In [44]:
files_list[3]

'datasets/DDD-teamblind.com.csv'

In [45]:
ddd_teamblind = pd.read_csv(files_list[3])
ddd_teamblind.head()

,Unnamed: 0,name,website,locations,size,industry,founded
0,0,Proteus Digital Health Careers,www.proteus.com,"Redwood City, CA",201 to 500 employees,Pharmaceuticals,2004
1,1,Zelis Careers,www.zelis.com,"Bedminster, NJ","1,001 to 5,000 employees",Information Technology & Services,2016
2,2,UpNest Careers,www.upnest.com,"Burlingame, CA",1 to 50 employees,Real Estate,2013
3,3,conferacity Careers,conferacity.com,"Menlo Park, CA",1 to 50 employees,Broadcast & Online Media,2013
4,4,Zenoti Careers,www.zenoti.com,"Bellevue, WA","501 to 1,000 employees",Computer Software,2010


In [46]:
to_drop, _ = columns_to_remove_not_remove(ddd_teamblind)

Columns to remove:
Unnamed: 0


In [47]:
ddd_teamblind = ddd_teamblind.drop(to_drop, axis=1)
to_rename = rename_columns(ddd_teamblind.columns)
ddd_teamblind.columns = to_rename
ddd_teamblind.head()

Columns to rename:
website -> links
locations -> country
size -> employees


,name,links,country,employees,industry,founded
0,Proteus Digital Health Careers,www.proteus.com,"Redwood City, CA",201 to 500 employees,Pharmaceuticals,2004
1,Zelis Careers,www.zelis.com,"Bedminster, NJ","1,001 to 5,000 employees",Information Technology & Services,2016
2,UpNest Careers,www.upnest.com,"Burlingame, CA",1 to 50 employees,Real Estate,2013
3,conferacity Careers,conferacity.com,"Menlo Park, CA",1 to 50 employees,Broadcast & Online Media,2013
4,Zenoti Careers,www.zenoti.com,"Bellevue, WA","501 to 1,000 employees",Computer Software,2010


In [48]:
cleaned_files.append(ddd_teamblind)

## FR

### FR_campaignindia.csv

In [49]:
files_list[4]

'datasets/FR_campaignindia.csv'

In [50]:
fr_india = pd.read_csv(files_list[4])
fr_india.head()

,RANK,BRAND NAME,CATEGORY,MAIN MARKET,2010 RANK
0,1,Sony,Consumer electronics,Regional,1
1,2,Samsung,Electronics/White goods,Regional,2
2,3,Panasonic,Consumer electronics,Regional,3
3,4,LG,Electronics/White goods,Regional,4
4,5,Canon,Camera/Office equipment,Regional,5


In [51]:
to_drop, _ = columns_to_remove_not_remove(fr_india)

Columns to remove:
RANK
MAIN MARKET
2010 RANK


In [52]:
fr_india = fr_india.drop(to_drop, axis=1)
to_rename = rename_columns(fr_india.columns)
fr_india.columns = to_rename
fr_india.head()

Columns to rename:
BRAND NAME -> name
CATEGORY -> industry


,name,industry
0,Sony,Consumer electronics
1,Samsung,Electronics/White goods
2,Panasonic,Consumer electronics
3,LG,Electronics/White goods
4,Canon,Camera/Office equipment


In [53]:
cleaned_files.append(fr_india)

### FR_company_social_urls.csv

In [54]:
files_list[5]

'datasets/FR_company_social_urls.csv'

In [55]:
fr_social_urls = pd.read_csv(files_list[5])
fr_social_urls.head()

,Company,Facebook,Twitter,Instagram,Pinterest
0,The Hershey Company,https://www.facebook.com/HersheyCompany/,https://twitter.com/hersheycompany,https://www.instagram.com/hersheycompany/,https://www.pinterest.com/hersheycompany/
1,ITC,https://it-it.facebook.com/ITCooperativa/,https://twitter.com/search?q=%23itc,https://www.instagram.com/tgruiz24/?hl=it,https://www.pinterest.it/itcaccessories/
2,Pgnig Group,https://www.facebook.com/PGNiG/,https://twitter.com/search?q=%23pgnig,https://www.instagram.com/pgnig_od/,NaN
3,MTN Group,https://m.facebook.com/profile.php?id=10859005...,https://twitter.com/mtngroup,https://www.instagram.com/mtn.company/?hl=it,https://www.pinterest.it/mtn_company/
4,Compass Group,https://it-it.facebook.com/people/Compass-Grou...,https://twitter.com/compassgroupuk,https://www.instagram.com/compassgroupse/?hl=it,https://www.pinterest.it/pin/288652657357881873/


In [56]:
url_dataset = ["Facebook", "Twitter", "Instagram", "Pinterest"]
fr_social_urls['link'] = fr_social_urls[url_dataset].values.tolist()

In [57]:
to_drop, _ = columns_to_remove_not_remove(fr_social_urls)
to_drop.extend(url_dataset)

Columns to remove:


In [58]:
fr_social_urls = fr_social_urls.drop(to_drop, axis=1)
to_rename = rename_columns(fr_social_urls.columns)
fr_social_urls.columns = to_rename
fr_social_urls.head()

Columns to rename:
Company -> name
link -> links


,name,links
0,The Hershey Company,"[https://www.facebook.com/HersheyCompany/, htt..."
1,ITC,"[https://it-it.facebook.com/ITCooperativa/, ht..."
2,Pgnig Group,"[https://www.facebook.com/PGNiG/, https://twit..."
3,MTN Group,[https://m.facebook.com/profile.php?id=1085900...
4,Compass Group,[https://it-it.facebook.com/people/Compass-Gro...


In [59]:
cleaned_files.append(fr_social_urls)

### FR_disfold.csv

In [119]:
files_list[6]

'datasets/FR_disfold.csv'

In [203]:
fr_disfold = pd.read_csv(files_list[6])
fr_disfold.head()

,Founded,Top Competitor,Employees,CEO,Name
0,"Founded: April 1, 1976",Samsung,"Employees: 100,000",CEO: Mr. Timothy D. Cook,Apple
1,"Founded: April 4, 1975",Oracle,"Employees: 181,000",CEO: Mr. Satya Nadella,Microsoft
2,NaN,ExxonMobil,NaN,CEO: Mr. Amin H. Nasser,Aramco
3,"Founded: October 2, 2015",Meta,"Employees: 156,500",CEO: Mr. Sundar Pichai,Alphabet
4,"Founded: July 5, 1994",Alibaba,"Employees: 1,608,000",CEO: Mr. Andrew R. Jassy,Amazon


In [204]:
to_drop, _ = columns_to_remove_not_remove(fr_disfold)

Columns to remove:
Top Competitor


In [205]:
fr_disfold = fr_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(fr_disfold.columns)
fr_disfold.columns = to_rename
fr_disfold.head()

Columns to rename:


,founded,employees,ceo,name
0,"Founded: April 1, 1976","Employees: 100,000",CEO: Mr. Timothy D. Cook,Apple
1,"Founded: April 4, 1975","Employees: 181,000",CEO: Mr. Satya Nadella,Microsoft
2,NaN,NaN,CEO: Mr. Amin H. Nasser,Aramco
3,"Founded: October 2, 2015","Employees: 156,500",CEO: Mr. Sundar Pichai,Alphabet
4,"Founded: July 5, 1994","Employees: 1,608,000",CEO: Mr. Andrew R. Jassy,Amazon


In [207]:
fr_disfold.founded = fr_disfold.founded.str.extract(r'([\d]{4})')
fr_disfold.ceo = fr_disfold.ceo.apply(lambda x : str(x).replace('CEO: ', ""))
fr_disfold.employees = fr_disfold.employees.apply(lambda x : str(x).replace('Employees: ', '').replace(",", ""))
fr_disfold = fr_disfold.replace('nan', np.nan)

fr_disfold.founded = fr_disfold.founded.astype(object).astype('Int64')
fr_disfold.employees = fr_disfold.employees.astype(object).astype('Int64')
fr_disfold.head()

,founded,employees,ceo,name
0,1976,100000,Mr. Timothy D. Cook,Apple
1,1975,181000,Mr. Satya Nadella,Microsoft
2,<NA>,<NA>,Mr. Amin H. Nasser,Aramco
3,2015,156500,Mr. Sundar Pichai,Alphabet
4,1994,1608000,Mr. Andrew R. Jassy,Amazon


In [64]:
cleaned_files.append(fr_disfold)

### FR_famcap_germany.csv

In [65]:
files_list[7]

'datasets/FR_famcap_germany.csv'

In [236]:
fr_germany = pd.read_csv(files_list[7])
fr_germany.head()

,Rank,Company,Revenues 2018 $m,Employees,Family ownership,Founded,Public/Private,Generation,Family executive,Woman in leadership,Sector,Headquarters,State (abbreviation),Website
0,1,Volkswagen Group,"266,315.1","665,260","Porsche, Piech",1937,Public,4,No,Yes,Automotive,Wolfsburg,NI,www.volkswagenag.de
1,2,"Schwarz Group (aka Lidl, Kaufland, ...)","119,418.4","429,000",Schwarz,1930,Private,2,No,No,Retail,Neckarsulm,BW,www.lidl.de
2,3,BMW Group,"109,018.3","134,682","Quandt, Klatten",1916,Public,4,No,Yes,Automotive,München,BY,www.bmwgroup.com
3,4,ALDI Discounter (Nord + Süd),"100,423.6","208,000",Albrecht,1913,Private,4,No,Yes,Retail,E en and Mülheim an der Ruhr,NW,www.aldi.de
4,5,Robert Bosch Group,"87,935.7","409,881",Bosch,1886,Private,3,No,No,Automotive,Gerlingen-Schillerhöhe,BW,www.bosch.de


In [226]:
to_drop, _ = columns_to_remove_not_remove(fr_germany)

Columns to remove:
Rank
Family ownership
Public/Private
Generation
Family executive
Woman in leadership
State (abbreviation)


In [237]:
fr_germany = fr_germany.drop(to_drop, axis=1)
to_rename = rename_columns(fr_germany.columns)
fr_germany.columns = to_rename
fr_germany.head()

Columns to rename:
Company -> name
Revenues 2018 $m -> revenue
Column error: Sector
Headquarters -> country
Website -> links


,name,revenue,employees,founded,sector,country,links
0,Volkswagen Group,"266,315.1","665,260",1937,Automotive,Wolfsburg,www.volkswagenag.de
1,"Schwarz Group (aka Lidl, Kaufland, ...)","119,418.4","429,000",1930,Retail,Neckarsulm,www.lidl.de
2,BMW Group,"109,018.3","134,682",1916,Automotive,München,www.bmwgroup.com
3,ALDI Discounter (Nord + Süd),"100,423.6","208,000",1913,Retail,E en and Mülheim an der Ruhr,www.aldi.de
4,Robert Bosch Group,"87,935.7","409,881",1886,Automotive,Gerlingen-Schillerhöhe,www.bosch.de


In [244]:
fr_germany.revenue = fr_germany.revenue.apply(lambda x : float(str(x).replace(',', '')))
fr_germany.employees = fr_germany.employees.apply(lambda x : str(x).replace(',', ''))

fr_germany.revenue = fr_germany.revenue.astype(float).astype(int)
fr_germany.employees = fr_germany.employees.astype(object).astype('Int64')
fr_germany.founded = fr_germany.founded.astype(object).astype('Int64')
fr_germany.head()

,name,revenue,employees,founded,sector,country,links
0,Volkswagen Group,266315,665260,1937,Automotive,Wolfsburg,www.volkswagenag.de
1,"Schwarz Group (aka Lidl, Kaufland, ...)",119418,429000,1930,Retail,Neckarsulm,www.lidl.de
2,BMW Group,109018,134682,1916,Automotive,München,www.bmwgroup.com
3,ALDI Discounter (Nord + Süd),100423,208000,1913,Retail,E en and Mülheim an der Ruhr,www.aldi.de
4,Robert Bosch Group,87935,409881,1886,Automotive,Gerlingen-Schillerhöhe,www.bosch.de


In [69]:
cleaned_files.append(fr_germany)

### FR_followers.csv

In [70]:
files_list[8]

'datasets/FR_followers.csv'

In [71]:
fr_followers = pd.read_csv(files_list[8])
fr_followers.head()

,Company,Facebook,Twitter,Instagram,Pinterest
0,Apple,14.031.481 persone seguono questa Pagina,"8,8 Mln",28.9M follower,-
1,Amazon,5.017.069 persone seguono questa Pagina,5 Mln,3.8M follower,"712,6mila follower"
2,China Construction Bank,-,-,-,-
3,Agricultural Bank of China,-,-,-,-
4,ICBC,15.811 persone seguono questa Pagina,22.516,"3,868 follower",-


#### Dataset da non usare?

### FR_forbes.csv

In [72]:
files_list[9]

'datasets/FR_forbes.csv'

In [254]:
fr_forbes = pd.read_csv(files_list[9])
fr_forbes.head()

,Industry,Founded,Revenue,Name,Location
0,Diversified Financials,1939.0,$276.1B,Berkshire Hathaway,"Omaha, Nebraska"
1,Banking,1984.0,$208.1B,ICBC,"Beijing, China"
2,Oil & Gas Operations,1933.0,$400.4B,Saudi Arabian Oil Company (Saudi Aramco),"Dhahran, Saudi Arabia"
3,Banking and Financial Services,2000.0,$124.5B,JPMorgan Chase,"New York, New York"
4,Banking,2014.0,$202.1B,China Construction Bank,"Beijing, China"


#### Rimuovere .0 da founded e sistemare location

In [255]:
to_rename = rename_columns(fr_forbes.columns)
fr_forbes.columns = to_rename
fr_forbes.head()

Columns to rename:
Location -> country


,industry,founded,revenue,name,country
0,Diversified Financials,1939.0,$276.1B,Berkshire Hathaway,"Omaha, Nebraska"
1,Banking,1984.0,$208.1B,ICBC,"Beijing, China"
2,Oil & Gas Operations,1933.0,$400.4B,Saudi Arabian Oil Company (Saudi Aramco),"Dhahran, Saudi Arabia"
3,Banking and Financial Services,2000.0,$124.5B,JPMorgan Chase,"New York, New York"
4,Banking,2014.0,$202.1B,China Construction Bank,"Beijing, China"


In [259]:
fr_forbes.founded = fr_forbes.founded.astype(str).replace(r'\.0$', '', regex=True)
fr_forbes.country = fr_forbes.country.str.split(',', n=1).str.get(-1)

fr_forbes.head()

,industry,founded,revenue,name,country
0,Diversified Financials,1939,$276.1B,Berkshire Hathaway,Nebraska
1,Banking,1984,$208.1B,ICBC,China
2,Oil & Gas Operations,1933,$400.4B,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia
3,Banking and Financial Services,2000,$124.5B,JPMorgan Chase,New York
4,Banking,2014,$202.1B,China Construction Bank,China


In [75]:
cleaned_files.append(fr_forbes)

### FR_ft.csv

In [76]:
files_list[10]

'datasets/FR_ft.csv'

In [261]:
fr_ft = pd.read_csv(files_list[10])
fr_ft.head()

,Rank,Name,in 2021 ranking,in 2020 ranking,Country,Sector,Absolute Growth Rate %,Compound Annual Growth Rate (CAGR) %,Revenue 2020 (€),Revenue 2017 (€),Number of employees 2020,Number of employees 2017,Founding Year
0,1,Swappie,No,No,Finland,Technology,"19,152.82",477.43,"97,611,814","507,000",218.0,1,2016
1,2,Kilo Health,No,No,Lithuania,Health,"16,541.73",450.05,"57,318,766","344,428",177.0,10,2013
2,3,OCI,No,No,UK,Financial Services,"13,133.13",409.59,"568,322,073","4,325,512",32.0,4,2012
3,4,OnlyFans,No,No,UK,Technology,"11,928.05",393.63,"316,732,986","2,652,185",800.0,4,2016
4,5,Enpal,No,No,Germany,Energy,"11,441.27",386.88,"56,109,613","486,165",365.0,9,2017


In [263]:
to_drop, _ = columns_to_remove_not_remove(fr_ft)

Columns to remove:
Rank
in 2021 ranking
in 2020 ranking
Absolute Growth Rate %
Compound Annual Growth Rate (CAGR) %
Revenue 2017 (€)
Number of employees 2017


In [264]:
fr_ft = fr_ft.drop(to_drop, axis=1)
to_rename = rename_columns(fr_ft.columns)
fr_ft.columns = to_rename
fr_ft.head()

Columns to rename:
Column error: Sector
Revenue 2020 (€) -> revenue
Number of employees 2020 -> employees
Founding Year -> founded


,name,country,sector,revenue,employees,founded
0,Swappie,Finland,Technology,"97,611,814",218.0,2016
1,Kilo Health,Lithuania,Health,"57,318,766",177.0,2013
2,OCI,UK,Financial Services,"568,322,073",32.0,2012
3,OnlyFans,UK,Technology,"316,732,986",800.0,2016
4,Enpal,Germany,Energy,"56,109,613",365.0,2017


In [269]:
fr_ft.revenue = fr_ft.revenue.apply(lambda x : (str(x).replace(',', '')))
fr_ft.employees = fr_ft.employees.astype(str).replace(r'\.0$', '', regex=True)
fr_ft = fr_ft.replace('nan', np.nan)

fr_ft.revenue = fr_ft.revenue.astype(object).astype('Int64')
fr_ft.employees = fr_ft.employees.astype(object).astype('Int64')
fr_ft.founded = fr_ft.founded.astype(object).astype('Int64')

fr_ft.head()

,name,country,sector,revenue,employees,founded
0,Swappie,Finland,Technology,97611814,218,2016
1,Kilo Health,Lithuania,Health,57318766,177,2013
2,OCI,UK,Financial Services,568322073,32,2012
3,OnlyFans,UK,Technology,316732986,800,2016
4,Enpal,Germany,Energy,56109613,365,2017


In [80]:
cleaned_files.append(fr_ft)

### FR_sole24ore.csv

In [81]:
files_list[11]

'datasets/FR_sole24ore.csv'

In [270]:
fr_sole24 = pd.read_csv(files_list[11])
fr_sole24.head()

,Unnamed: 0,address,country,sector,employees,name
0,0,Via Maurizio Gonzaga 5,NaN,E-commerce,7.337,Winelivery S.r.l.
1,1,Via Damiano Chiesa 53,Bitritto,Servizi generali,8.375,Pegaso Security S.r.l.
2,2,Piazza IV Novembre 4,Milano,Vendita al dettaglio & all'ingrosso,13.889,Euro Top Brand S.r.l.s.
3,3,Piazza Manifattura 1,Rovereto,Energia,3.585,Infinityhub S.p.A. Benefit
4,4,Via Solferino 36,Milano,Servizi finanziari,11.302,Credimi S.p.a.


In [271]:
to_drop, _ = columns_to_remove_not_remove(fr_sole24)
to_drop.append("address")

Columns to remove:
Unnamed: 0


In [272]:
fr_sole24 = fr_sole24.drop(to_drop, axis=1)
to_rename = rename_columns(fr_sole24.columns)
fr_sole24.columns = to_rename
fr_sole24.head()

Columns to rename:
Column error: sector


,country,sector,employees,name
0,NaN,E-commerce,7.337,Winelivery S.r.l.
1,Bitritto,Servizi generali,8.375,Pegaso Security S.r.l.
2,Milano,Vendita al dettaglio & all'ingrosso,13.889,Euro Top Brand S.r.l.s.
3,Rovereto,Energia,3.585,Infinityhub S.p.A. Benefit
4,Milano,Servizi finanziari,11.302,Credimi S.p.a.


In [273]:
fr_sole24.employees = fr_sole24.employees.apply(lambda x : (str(x).replace('.', '')))

fr_sole24.employees = fr_sole24.employees.astype(object).astype('Int64')
fr_sole24.head()

,country,sector,employees,name
0,NaN,E-commerce,7337,Winelivery S.r.l.
1,Bitritto,Servizi generali,8375,Pegaso Security S.r.l.
2,Milano,Vendita al dettaglio & all'ingrosso,13889,Euro Top Brand S.r.l.s.
3,Rovereto,Energia,3585,Infinityhub S.p.A. Benefit
4,Milano,Servizi finanziari,11302,Credimi S.p.a.


In [85]:
cleaned_files.append(fr_sole24)

### FR_valuetoday.csv

In [86]:
files_list[12]

'datasets/FR_valuetoday.csv'

In [275]:
fr_value = pd.read_csv(files_list[12])
fr_value.head()

,IPO Year,Number of Employees,Country,CEO,Founded Year,Market Value,Name
0,1980.0,"154,000",USA,Tim Cook,1976.0,"2,825.00 Billion USD",APPLE
1,1986.0,"220,000",USA,Satya Nadella,1975.0,"2,358.00 Billion USD",MICROSOFT CORPORATION
2,NaN,"79,800",Saudi Arabia,Amin H. Al-Nasser,1933.0,"1,889.60 Billion USD",SAUDI ARABIAN OIL COMPANY (Saudi Aramco)
3,2004.0,"156,500",USA,Sundar Pichai,2015.0,"1,820.00 Billion USD",ALPHABET
4,1997.0,"1,335,000",USA,Andy Jassy,1994.0,"1,649.00 Billion USD",AMAZON.COM


In [277]:
to_drop, _ = columns_to_remove_not_remove(fr_value)

Columns to remove:
IPO Year
Market Value


In [278]:
fr_value = fr_value.drop(to_drop, axis=1)
to_rename = rename_columns(fr_value.columns)
fr_value.columns = to_rename
fr_value.head()

Columns to rename:
Number of Employees -> employees
Founded Year -> founded


,employees,country,ceo,founded,name
0,"154,000",USA,Tim Cook,1976.0,APPLE
1,"220,000",USA,Satya Nadella,1975.0,MICROSOFT CORPORATION
2,"79,800",Saudi Arabia,Amin H. Al-Nasser,1933.0,SAUDI ARABIAN OIL COMPANY (Saudi Aramco)
3,"156,500",USA,Sundar Pichai,2015.0,ALPHABET
4,"1,335,000",USA,Andy Jassy,1994.0,AMAZON.COM


In [280]:
fr_value.founded = fr_value.founded.astype(str).replace(r'\.0$', '', regex=True)
fr_value.employees = fr_value.employees.apply(lambda x : (str(x).replace(',', '')))
fr_value = fr_value.replace('nan', np.nan)

fr_value.employees = fr_value.employees.astype(object).astype('Int64')
fr_value.founded = fr_value.founded.astype(object).astype('Int64')

fr_value.head()

,employees,country,ceo,founded,name
0,154000,USA,Tim Cook,1976,APPLE
1,220000,USA,Satya Nadella,1975,MICROSOFT CORPORATION
2,79800,Saudi Arabia,Amin H. Al-Nasser,1933,SAUDI ARABIAN OIL COMPANY (Saudi Aramco)
3,156500,USA,Sundar Pichai,2015,ALPHABET
4,1335000,USA,Andy Jassy,1994,AMAZON.COM


In [90]:
cleaned_files.append(fr_value)

## GioPonSpiz

### GioPonSpiz-companiesmarketcap.com.json

In [91]:
files_list[13]

'datasets/GioPonSpiz-companiesmarketcap.com.json'

In [92]:
gps_companies = pd.read_json(files_list[13])
gps_companies.head()

,name,codice,pricecap,price,country
0,Apple,AAPL,$2.143 T,$134.76,USA
1,Saudi Aramco,2222.SR,$1.878 T,$8.54,S. Arabia
2,Microsoft,MSFT,$1.783 T,$239.23,USA
3,Alphabet (Google),GOOG,$1.195 T,$92.80,USA
4,Amazon,AMZN,$1.000 T,$98.12,USA


In [93]:
to_drop, _ = columns_to_remove_not_remove(gps_companies)

Columns to remove:
codice
price


In [94]:
gps_companies = gps_companies.drop(to_drop, axis=1)
to_rename = rename_columns(gps_companies.columns)
gps_companies.columns = to_rename
gps_companies.head()

Columns to rename:
pricecap -> marketcap


,name,marketcap,country
0,Apple,$2.143 T,USA
1,Saudi Aramco,$1.878 T,S. Arabia
2,Microsoft,$1.783 T,USA
3,Alphabet (Google),$1.195 T,USA
4,Amazon,$1.000 T,USA


In [95]:
cleaned_files.append(gps_companies)

### GioPonSpiz-disfold.com.json

In [96]:
files_list[14]

'datasets/GioPonSpiz-disfold.com.json'

In [97]:
gps_disfold = pd.read_json(files_list[14])
gps_disfold.head()

,name,marketCap,stock,sector,industry,country
0,Apple Inc.,$2.825 T,AAPL,Technology,Consumer Electronics,United States
1,Microsoft Corporation,$2.328 T,MSFT,Technology,Software—Infrastructure,United States
2,Saudi Arabian Oil Company,$2.297 T,2222,Energy,Oil & Gas Integrated,Saudi Arabia
3,Alphabet Inc.,$1.825 T,GOOG,Communication Services,Internet Content & Information,United States
4,"Amazon.com, Inc.",$1.642 T,AMZN,Consumer Discretionary,Internet Retail,United States


In [98]:
to_drop, _ = columns_to_remove_not_remove(gps_disfold)

Columns to remove:
stock


In [99]:
gps_disfold = gps_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(gps_disfold.columns)
gps_disfold.columns = to_rename
gps_disfold.head()

Columns to rename:
Column error: sector


,name,marketcap,sector,industry,country
0,Apple Inc.,$2.825 T,Technology,Consumer Electronics,United States
1,Microsoft Corporation,$2.328 T,Technology,Software—Infrastructure,United States
2,Saudi Arabian Oil Company,$2.297 T,Energy,Oil & Gas Integrated,Saudi Arabia
3,Alphabet Inc.,$1.825 T,Communication Services,Internet Content & Information,United States
4,"Amazon.com, Inc.",$1.642 T,Consumer Discretionary,Internet Retail,United States


In [100]:
cleaned_files.append(gps_disfold)

### GioPonSpiz-value.today.json

In [101]:
files_list[15]

'datasets/GioPonSpiz-value.today.json'

In [102]:
gps_value = pd.read_json(files_list[15])
gps_value.head()

,name,world_rank,marketValue,marketCap,CEO,country
0,APPLE,1,"2,825.000 Billion USD","2,510.000 Billion USD",Tim Cook,USA
1,MICROSOFT CORPORATION,2,"2,358.000 Billion USD","2,269.000 Billion USD",None,USA
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),3,"1,889.600 Billion USD","1,891.600 Billion USD",H.E. Yasir Othman Al-Rumayyan,Saudi Arabia
3,ALPHABET,4,"1,820.000 Billion USD","1,934.000 Billion USD",John L. Hennessy,USA
4,AMAZON.COM,5,"1,649.000 Billion USD","1,758.000 Billion USD",Andy Jassy,USA


In [103]:
to_drop, _ = columns_to_remove_not_remove(gps_value)

Columns to remove:
world_rank
marketValue


In [104]:
gps_value = gps_value.drop(to_drop, axis=1)
to_rename = rename_columns(gps_value.columns)
gps_value.columns = to_rename
gps_value.head()

Columns to rename:


,name,marketcap,ceo,country
0,APPLE,"2,510.000 Billion USD",Tim Cook,USA
1,MICROSOFT CORPORATION,"2,269.000 Billion USD",None,USA
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),"1,891.600 Billion USD",H.E. Yasir Othman Al-Rumayyan,Saudi Arabia
3,ALPHABET,"1,934.000 Billion USD",John L. Hennessy,USA
4,AMAZON.COM,"1,758.000 Billion USD",Andy Jassy,USA


In [105]:
cleaned_files.append(gps_value)

### GioPonSpiz-wikipedia.org.json

In [106]:
files_list[16]

'datasets/GioPonSpiz-wikipedia.org.json'

In [107]:
gps_wiki = pd.read_json(files_list[16])
gps_wiki.head()

,Name,Industry,Sector,Headquarters,Founded
0,3H Biomedical,Health care,Biotechnology,Uppsala,2004
1,AarhusKarlshamn,Consumer goods,Food products,Malmö,2005
2,Abba Seafood,Consumer goods,Food products,Gothenburg,1883
3,ABU Garcia,Consumer goods,Recreational products,Svängsta,1921
4,Acando,Technology,Software,Stockholm,2001


In [108]:
to_rename = rename_columns(gps_wiki.columns)
gps_wiki.columns = to_rename
gps_wiki.founded = gps_wiki.founded.astype(object).astype('Int64')
gps_wiki.head()

Columns to rename:
Column error: Sector
Headquarters -> country


,name,industry,sector,country,founded
0,3H Biomedical,Health care,Biotechnology,Uppsala,2004
1,AarhusKarlshamn,Consumer goods,Food products,Malmö,2005
2,Abba Seafood,Consumer goods,Food products,Gothenburg,1883
3,ABU Garcia,Consumer goods,Recreational products,Svängsta,1921
4,Acando,Technology,Software,Stockholm,2001


In [109]:
cleaned_files.append(gps_wiki)

## MalPatSaj

### MalPatSaj-companiesmarketcap.com.xls

In [110]:
files_list[17]

'datasets/MalPatSaj-companiesmarketcap.com.xls'

In [111]:
mps_companies = pd.read_excel(files_list[17])
mps_companies.head()

,Unnamed: 0,Name,Symbol,MarketCap,Price,Country
0,0,Apple,AAPL,$2.278 T,$143.21,USA
1,1,Microsoft,MSFT,$1.917 T,$257.22,USA
2,2,Saudi Aramco,2222.SR,$1.810 T,$8.23,S. Arabia
3,3,Alphabet (Google),GOOG,$1.232 T,$95.31,USA
4,4,Amazon,AMZN,$934.27 B,$91.58,USA


In [112]:
to_drop, _ = columns_to_remove_not_remove(mps_companies)

Columns to remove:
Unnamed: 0
Symbol
Price


In [113]:
mps_companies = mps_companies.drop(to_drop, axis=1)
to_rename = rename_columns(mps_companies.columns)
mps_companies.columns = to_rename
mps_companies.head()

Columns to rename:


,name,marketcap,country
0,Apple,$2.278 T,USA
1,Microsoft,$1.917 T,USA
2,Saudi Aramco,$1.810 T,S. Arabia
3,Alphabet (Google),$1.232 T,USA
4,Amazon,$934.27 B,USA


In [114]:
cleaned_files.append(mps_companies)

### MalPatSaj-disfold.com.xlsx

In [115]:
files_list[18]

'datasets/MalPatSaj-disfold.com.xlsx'

In [116]:
mps_disfold = pd.read_excel(files_list[18])
mps_disfold.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Unnamed: 0,Name,MarketCap,Stock,Country,Sector,Industry
0,0,Apple Inc.,$2.825 T,AAPL,United States,Technology,Consumer Electronics
1,1,Microsoft Corporation,$2.328 T,MSFT,United States,Technology,Software—Infrastructure
2,2,Saudi Arabian Oil Company,$2.297 T,2222,Saudi Arabia,Energy,Oil & Gas Integrated
3,3,Alphabet Inc.,$1.825 T,GOOG,United States,Communication Services,Internet Content & Information
4,4,"Amazon.com, Inc.",$1.642 T,AMZN,United States,Consumer Discretionary,Internet Retail


In [117]:
to_drop, _ = columns_to_remove_not_remove(mps_disfold)

Columns to remove:
Unnamed: 0
Stock


In [118]:
mps_disfold = mps_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(mps_disfold.columns)
mps_disfold.columns = to_rename
mps_disfold.head()

Columns to rename:
Column error: Sector


,name,marketcap,country,sector,industry
0,Apple Inc.,$2.825 T,United States,Technology,Consumer Electronics
1,Microsoft Corporation,$2.328 T,United States,Technology,Software—Infrastructure
2,Saudi Arabian Oil Company,$2.297 T,Saudi Arabia,Energy,Oil & Gas Integrated
3,Alphabet Inc.,$1.825 T,United States,Communication Services,Internet Content & Information
4,"Amazon.com, Inc.",$1.642 T,United States,Consumer Discretionary,Internet Retail


In [119]:
cleaned_files.append(mps_disfold)

### MalPatSaj-forbes.com.xls

In [120]:
files_list[19]

'datasets/MalPatSaj-forbes.com.xls'

In [121]:
mps_forbes = pd.read_excel(files_list[19])
mps_forbes.head()

,Unnamed: 0,Name,Country,Sales,Profit,Assets,Market Value
0,0,Berkshire Hathaway,United States,$276.09 B,$89.8 B,$958.78 B,$741.48 B
1,1,ICBC,China,$208.13 B,$54.03 B,"$5,518.51 B",$214.43 B
2,2,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$105.36 B,$576.04 B,"$2,292.08 B"
3,3,JPMorgan Chase,United States,$124.54 B,$42.12 B,"$3,954.69 B",$374.45 B
4,4,China Construction Bank,China,$202.07 B,$46.89 B,"$4,746.95 B",$181.32 B


In [122]:
to_drop, _ = columns_to_remove_not_remove(mps_forbes)

Columns to remove:
Unnamed: 0
Profit
Market Value


In [123]:
mps_forbes = mps_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(mps_forbes.columns)
mps_forbes.columns = to_rename
mps_forbes.head()

Columns to rename:
Sales -> revenue
Assets -> revenue


,name,country,revenue,revenue
0,Berkshire Hathaway,United States,$276.09 B,$958.78 B
1,ICBC,China,$208.13 B,"$5,518.51 B"
2,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$576.04 B
3,JPMorgan Chase,United States,$124.54 B,"$3,954.69 B"
4,China Construction Bank,China,$202.07 B,"$4,746.95 B"


In [124]:
cleaned_files.append(mps_forbes)

### MalPatSaj-wikipedia.org.xls

In [125]:
files_list[20]

'datasets/MalPatSaj-wikipedia.org.xls'

In [126]:
mps_wiki = pd.read_excel(files_list[20])
mps_wiki.head()

,Unnamed: 0,Name,Industry,Sector,Headquarters,Founded,Notes
0,0,Agrana,Consumer goods,Food products,Vienna,1988,"Sugar, starch, fruit juice concentrates and fr..."
1,1,pewag group,Industrials,Industrial machinery,Klagenfurt,1479,Chain manufacturer
2,2,AHT Cooling Systems,Industrials,Electronic equipment,Rottenmann,1983,Cooling and refrigeration
3,3,Ammerer Bed Company,Consumer goods,Durable household products,Ried im Innkreis,1763,NaN
4,4,ams AG,Technology,Semiconductors,Premstätten,1981,Semiconductors


In [127]:
to_drop, _ = columns_to_remove_not_remove(mps_wiki)

Columns to remove:
Unnamed: 0
Notes


In [128]:
mps_wiki = mps_wiki.drop(to_drop, axis=1)
to_rename = rename_columns(mps_wiki.columns)
mps_wiki.columns = to_rename
mps_wiki.founded = mps_wiki.founded.astype(object).astype('Int64')
mps_wiki.head()

Columns to rename:
Column error: Sector
Headquarters -> country


,name,industry,sector,country,founded
0,Agrana,Consumer goods,Food products,Vienna,1988
1,pewag group,Industrials,Industrial machinery,Klagenfurt,1479
2,AHT Cooling Systems,Industrials,Electronic equipment,Rottenmann,1983
3,Ammerer Bed Company,Consumer goods,Durable household products,Ried im Innkreis,1763
4,ams AG,Technology,Semiconductors,Premstätten,1981


In [129]:
cleaned_files.append(mps_wiki)

## MarScoToc

### MarScoToc-ambitionbox.com.csv

In [130]:
files_list[21]

'datasets/MarScoToc-ambitionbox.com.csv'

In [296]:
mst_ambition = pd.read_csv(files_list[21], encoding="utf-8", encoding_errors="ignore")
mst_ambition.head()

,Name,Industry,Headquarter,Ownership,Foundation Year
0,TCS,IT Services & Consulting,"Mumbai, Maharashtra, India",Public,1968 (55 yrs old)
1,Accenture,IT Services & Consulting,"Dublin, Ireland",Public,1989 (34 yrs old)
2,Cognizant,IT Services & Consulting,"Teaneck. New Jersey., United States (USA)",Private,1994 (29 yrs old)
3,ICICI Bank,Banking,"Mumbai, Maharashtra, India",Public,1994 (29 yrs old)
4,HDFC Bank,Banking,"Mumbai, Maharashtra, India",Public,1994 (29 yrs old)


In [283]:
to_drop, _ = columns_to_remove_not_remove(mst_ambition)

Columns to remove:
Ownership


In [297]:
mst_ambition = mst_ambition.drop(to_drop, axis=1)
to_rename = rename_columns(mst_ambition.columns)
mst_ambition.columns = to_rename
mst_ambition.head()

Columns to rename:
Headquarter -> country
Foundation Year -> founded


,name,industry,country,founded
0,TCS,IT Services & Consulting,"Mumbai, Maharashtra, India",1968 (55 yrs old)
1,Accenture,IT Services & Consulting,"Dublin, Ireland",1989 (34 yrs old)
2,Cognizant,IT Services & Consulting,"Teaneck. New Jersey., United States (USA)",1994 (29 yrs old)
3,ICICI Bank,Banking,"Mumbai, Maharashtra, India",1994 (29 yrs old)
4,HDFC Bank,Banking,"Mumbai, Maharashtra, India",1994 (29 yrs old)


In [298]:
mst_ambition.country = mst_ambition.country.apply(lambda x: str(x).split(',')[-1])
mst_ambition.founded = mst_ambition.founded.str.extract(r'([\d]{4})')
mst_ambition = mst_ambition.replace('nan', np.nan)

mst_ambition.founded = mst_ambition.founded.astype(object).astype('Int64')

mst_ambition.head()

,name,industry,country,founded
0,TCS,IT Services & Consulting,India,1968
1,Accenture,IT Services & Consulting,Ireland,1989
2,Cognizant,IT Services & Consulting,United States (USA),1994
3,ICICI Bank,Banking,India,1994
4,HDFC Bank,Banking,India,1994


In [134]:
cleaned_files.append(mst_ambition)

### MarScoToc-companiesmarketcap.com.csv

In [135]:
files_list[22]

'datasets/MarScoToc-companiesmarketcap.com.csv'

In [136]:
mst_companies = pd.read_csv(files_list[22], encoding="utf-8", encoding_errors="ignore")
mst_companies.head()

,Name,Master Cap,Country,Rank,Share Price
0,Apple,$2.183 T,USA,#1,$137.87
1,Saudi Aramco,$1.887 T,Saudi Arabia,#2,$8.58
2,Microsoft,$1.790 T,USA,#3,$240.22
3,Alphabet (Google),$1.278 T,USA,#4,$99.28
4,Amazon,$992.11 B,USA,#5,$97.25


In [137]:
to_drop, _ = columns_to_remove_not_remove(mst_companies)

Columns to remove:
Rank
Share Price


In [138]:
mst_companies = mst_companies.drop(to_drop, axis=1)
to_rename = rename_columns(mst_companies.columns)
mst_companies.columns = to_rename
mst_companies.head()

Columns to rename:
Master Cap -> marketcap


,name,marketcap,country
0,Apple,$2.183 T,USA
1,Saudi Aramco,$1.887 T,Saudi Arabia
2,Microsoft,$1.790 T,USA
3,Alphabet (Google),$1.278 T,USA
4,Amazon,$992.11 B,USA


In [139]:
cleaned_files.append(mst_companies)

### MarScoToc-wikipedia.org.csv

In [140]:
files_list[24]

'datasets/MarScoToc-wikipedia.org.csv'

In [299]:
mst_wiki = pd.read_csv(files_list[24], encoding="utf-8", encoding_errors="ignore")
mst_wiki.head()

,Name,Industry,headQuarters,Founded,Area Served
0,85C Bakery Cafe,Food and Beverage,Taipei,2004; 18 years ago (2004),Worldwide
1,10.or,Smartphones,Taipei,2004; 18 years ago (2004),Worldwide
2,2U (company),Smartphones,"Lanham, Maryland",2008; 14 years ago (2008),Global
3,3i,Corporate finance,London,1945; 77 years ago (1945),NaN
4,3M,Conglomerate,London,"June 13, 1902; 120 years ago (1902-06-13) in T...",Worldwide


In [301]:
to_drop, _ = columns_to_remove_not_remove(mst_wiki)

Columns to remove:
Area Served


In [302]:
mst_wiki = mst_wiki.drop(to_drop, axis=1)
to_rename = rename_columns(mst_wiki.columns)
mst_wiki.columns = to_rename
mst_wiki.head()

Columns to rename:
headQuarters -> country


,name,industry,country,founded
0,85C Bakery Cafe,Food and Beverage,Taipei,2004; 18 years ago (2004)
1,10.or,Smartphones,Taipei,2004; 18 years ago (2004)
2,2U (company),Smartphones,"Lanham, Maryland",2008; 14 years ago (2008)
3,3i,Corporate finance,London,1945; 77 years ago (1945)
4,3M,Conglomerate,London,"June 13, 1902; 120 years ago (1902-06-13) in T..."


In [303]:
mst_wiki.country = mst_wiki.country.apply(lambda x: str(x).split(',')[-1])
mst_wiki.founded = mst_wiki.founded.str.extract(r'([\d]{4})')
mst_wiki = mst_wiki.replace('nan', np.nan)

mst_wiki.founded = mst_wiki.founded.astype(object).astype('Int64')

mst_wiki.head()

,name,industry,country,founded
0,85C Bakery Cafe,Food and Beverage,Taipei,2004
1,10.or,Smartphones,Taipei,2004
2,2U (company),Smartphones,Maryland,2008
3,3i,Corporate finance,London,1945
4,3M,Conglomerate,London,1902


In [144]:
cleaned_files.append(mst_wiki)

## avengers

### avengers-companiesmarketcap.jsonl

In [145]:
files_list[25]

'datasets/avengers-companiesmarketcap.jsonl'

In [146]:
avengers_companies = pd.read_json(files_list[25], lines=True)
avengers_companies.head()

,id,name,rank,market_cap,country,share_price,change_1_day,change_1_year,categories
0,1aeb78c70807445290795f145210a39e,Apple,1,$2.351 T,USA,$147.81,-0.34%,-9.74%,"[Tech, Software, Dow jones, Tech Hardware, Ele..."
1,8443bd488b2543718e4dab19e7d1b371,Bristol-myers Squibb,57,$172.49 B,USA,$81.13,0.31%,46.42%,"[Pharmaceuticals, Biotech]"
2,d35ff63680a645f986048ab48d2fe322,Wells Fargo,56,$175.05 B,USA,$45.94,-1.98%,-6.07%,"[Banks, Financial services]"
3,0664af11c7b9417bab58542ffa880c0e,Nike,55,$175.54 B,USA,$112.20,1.29%,-34.00%,"[Sports goods, Footwear, Dow jones, Clothing]"
4,4a67737ea89e450b8e8677a91e90327f,Walt Disney,54,$177.08 B,USA,$99.43,0.85%,-32.45%,"[Entertainment, Dow jones]"


In [147]:
to_drop, _ = columns_to_remove_not_remove(avengers_companies)

Columns to remove:
id
rank
share_price
change_1_day
change_1_year


In [148]:
avengers_companies = avengers_companies.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_companies.columns)
avengers_companies.columns = to_rename
avengers_companies.head()

Columns to rename:
market_cap -> marketcap
categories -> industry


,name,marketcap,country,industry
0,Apple,$2.351 T,USA,"[Tech, Software, Dow jones, Tech Hardware, Ele..."
1,Bristol-myers Squibb,$172.49 B,USA,"[Pharmaceuticals, Biotech]"
2,Wells Fargo,$175.05 B,USA,"[Banks, Financial services]"
3,Nike,$175.54 B,USA,"[Sports goods, Footwear, Dow jones, Clothing]"
4,Walt Disney,$177.08 B,USA,"[Entertainment, Dow jones]"


In [149]:
cleaned_files.append(avengers_companies)

### avengers-disfold.jsonl

In [150]:
files_list[26]

'datasets/avengers-disfold.jsonl'

In [312]:
avengers_disfold = pd.read_json(files_list[26], lines=True)
avengers_disfold.head()

,id,name,official_name,headquarters_country,headquarters_continent,founded,employees,ceo,market_cap,categories,gbp
0,1e4dfb844d5a47e39c14699648cbd55a,Apple,Apple Inc.,Cupertino,CA,"April 1, 1976","100,000",Mr. Timothy D. Cook,$2.825 Trillion,"[UnitedStates, Technology, ConsumerElectronics...",NaN
1,031f69c6c7fd49d89b00ecdfcd294bad,Danaher,Danaher Corporation,Washington,DC,1969,"78,000",Mr. Rainer M. Blair,$208.03 Billion,"[UnitedStates, Healthcare, Diagnostics&Researc...",NaN
2,8acd1acfe59d4fc88c28038c9d3e8911,Merck,"Merck & Co., Inc.",Kenilworth,NJ,1891,"67,000",Mr. Robert M. Davis J.D.,$210.38 Billion,"[UnitedStates, Healthcare, DrugManufacturers—G...",NaN
3,ba6efa594766439dafb4045f0b25a23a,Bhp,BHP Group,Melbourne,VIC,1885,"40,110","Mr. Mike P. Henry B.Sc., BSc (Chem)",A$281.66 Billion,"[Australia, BasicMaterials, OtherIndustrialMet...",$211.25 Billion
4,83d47a2c8bd545cf80c17abfa6d81cf9,Nike,"Nike, Inc.",Beaverton,OR,"January 25, 1964","73,300",Mr. John J. Donahoe II,$213.60 Billion,"[UnitedStates, ConsumerDiscretionary, Footwear...",NaN


In [313]:
avengers_disfold.gbp = avengers_disfold.gbp.fillna(avengers_disfold.market_cap)

In [314]:
to_drop, _ = columns_to_remove_not_remove(avengers_disfold)
to_drop.append("market_cap")

Columns to remove:
id
official_name
headquarters_continent


In [315]:
avengers_disfold = avengers_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_disfold.columns)
avengers_disfold.columns = to_rename
avengers_disfold.head()

Columns to rename:
headquarters_country -> country
categories -> industry
gbp -> marketcap


,name,country,founded,employees,ceo,industry,marketcap
0,Apple,Cupertino,"April 1, 1976","100,000",Mr. Timothy D. Cook,"[UnitedStates, Technology, ConsumerElectronics...",$2.825 Trillion
1,Danaher,Washington,1969,"78,000",Mr. Rainer M. Blair,"[UnitedStates, Healthcare, Diagnostics&Researc...",$208.03 Billion
2,Merck,Kenilworth,1891,"67,000",Mr. Robert M. Davis J.D.,"[UnitedStates, Healthcare, DrugManufacturers—G...",$210.38 Billion
3,Bhp,Melbourne,1885,"40,110","Mr. Mike P. Henry B.Sc., BSc (Chem)","[Australia, BasicMaterials, OtherIndustrialMet...",$211.25 Billion
4,Nike,Beaverton,"January 25, 1964","73,300",Mr. John J. Donahoe II,"[UnitedStates, ConsumerDiscretionary, Footwear...",$213.60 Billion


In [316]:
avengers_disfold.employees = avengers_disfold.employees.apply(lambda x: str(x).replace(',', ''))
avengers_disfold.founded = avengers_disfold.founded.str.extract(r'([\d]{4})')
avengers_disfold = avengers_disfold.replace('nan', np.nan)

avengers_disfold.employees = avengers_disfold.employees.astype(object).astype('Int64')
avengers_disfold.founded = avengers_disfold.founded.astype(object).astype('Int64')

avengers_disfold.head()

,name,country,founded,employees,ceo,industry,marketcap
0,Apple,Cupertino,1976,100000,Mr. Timothy D. Cook,"[UnitedStates, Technology, ConsumerElectronics...",$2.825 Trillion
1,Danaher,Washington,1969,78000,Mr. Rainer M. Blair,"[UnitedStates, Healthcare, Diagnostics&Researc...",$208.03 Billion
2,Merck,Kenilworth,1891,67000,Mr. Robert M. Davis J.D.,"[UnitedStates, Healthcare, DrugManufacturers—G...",$210.38 Billion
3,Bhp,Melbourne,1885,40110,"Mr. Mike P. Henry B.Sc., BSc (Chem)","[Australia, BasicMaterials, OtherIndustrialMet...",$211.25 Billion
4,Nike,Beaverton,1964,73300,Mr. John J. Donahoe II,"[UnitedStates, ConsumerDiscretionary, Footwear...",$213.60 Billion


In [154]:
cleaned_files.append(avengers_disfold)

### avengers-hithorizons.jsonl

In [155]:
files_list[27]

'datasets/avengers-hithorizons.jsonl'

In [326]:
avengers_hit = pd.read_json(files_list[27], lines=True)
avengers_hit.head()

,id,name,address,nation,hhid,industry,sic_code,type,est_of_ownership
0,d60823f20d864692b517a8ad6c1418ed,Enel Spa,"VIALE REGINA MARGHERITA 137, ROMA, 00198, ROMA...",IT00934061003,H-IT0031426662,"Finance, Insurance, and Real Estate",Holding company (6719),Corporation,1962
1,c7856532ca8b4dd286a9602c77221035,Esselunga Spa,"VIA GIAMBOLOGNA 1, PIOLTELLO, 20096, MILANO, I...",IT04916380159,H-IT0068388950,Retail Trade,Ret mail-order house (5961),Corporation,1957
2,e8e04ae3e4594e3b836e62077155b19a,Superit Srl,"VIA VITTOR PISANI 20, MILANO, 20124, MILANO, I...",IT11068950960,H-IT0289899385,"Finance, Insurance, and Real Estate",Holding company (6719),Corporation,2019
3,37290f39faa54935813c0c6f840d7353,Saras Spa,"GALLERIA PASSARELLA 2, MILANO, 20122, MILANO, ...",IT00136440922,H-IT0018414451,Manufacturing,Petroleum refiner (2911),Corporation,1962
4,bb3a80a40f704c688f804f627667fd31,Unicredit Spa,"PIAZZA GAE AULENTI 3 TOWER A, MILANO, 20154, M...",IT00348170101,H-IT0009571558,"Finance, Insurance, and Real Estate",National commercial bank (6021),Corporation,1870


In [319]:
to_drop, _ = columns_to_remove_not_remove(avengers_hit)

Columns to remove:
id
nation
hhid
sic_code
type


In [327]:
avengers_hit = avengers_hit.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_hit.columns)
avengers_hit.columns = to_rename
avengers_hit.head()

Columns to rename:
address -> country
est_of_ownership -> founded


,name,country,industry,founded
0,Enel Spa,"VIALE REGINA MARGHERITA 137, ROMA, 00198, ROMA...","Finance, Insurance, and Real Estate",1962
1,Esselunga Spa,"VIA GIAMBOLOGNA 1, PIOLTELLO, 20096, MILANO, I...",Retail Trade,1957
2,Superit Srl,"VIA VITTOR PISANI 20, MILANO, 20124, MILANO, I...","Finance, Insurance, and Real Estate",2019
3,Saras Spa,"GALLERIA PASSARELLA 2, MILANO, 20122, MILANO, ...",Manufacturing,1962
4,Unicredit Spa,"PIAZZA GAE AULENTI 3 TOWER A, MILANO, 20154, M...","Finance, Insurance, and Real Estate",1870


In [337]:
avengers_hit.country = avengers_hit.country.apply(lambda x: str(x).split(',', 5)[-1])
avengers_hit.founded = avengers_hit.founded.astype(object).astype('Int64')

avengers_hit.head()

,name,country,industry,founded
0,Enel Spa,ROMA,"Finance, Insurance, and Real Estate",1962
1,Esselunga Spa,MILANO,Retail Trade,1957
2,Superit Srl,MILANO,"Finance, Insurance, and Real Estate",2019
3,Saras Spa,MILANO,Manufacturing,1962
4,Unicredit Spa,MILANO,"Finance, Insurance, and Real Estate",1870


In [159]:
cleaned_files.append(avengers_hit)

### avengers-valuetoday.jsonl

In [160]:
files_list[28]

'datasets/avengers-valuetoday.jsonl'

In [161]:
avengers_value = pd.read_json(files_list[28], lines=True)
avengers_value.head()

,id,name,world_rank,annual_revenue_in_usd,annual_net_income_in_usd,annual_results_for_year_ending,total_assets_in_usd,total_liabilities_in_usd,total_equity_in_usd,headquarters_region_city,headquarters_country,headquarters_sub_region,headquarters_continent,company_business,number_of_employees,ceo,founders,company_website
0,b9a098214d1a4ce1acc834cf17aef660,Apple,1,"394,330 Million USD","99,803 Million USD",Sep-2022,"352,760 Million USD","302,080 Million USD","50,672 Million USD",California,USA,Northern America,Americas,"[Technology, Mobiles & Accessories, Electronic...","154,000",Tim Cook,"[Steve Jobs, Steve Wozniak, Ronald Wayne]",https://www.apple.com/
1,42f1b7d5a40343de9837fc75d74767c3,Taiwan Semiconductor Manufacturing Company (tsmc),10,"55,306 Million USD","20,783 Million USD",Dec-2021,"129,800 Million USD","54,253 Million USD","75,543 Million USD",Hsinchu,Taiwan,South East Asia,Asia,"[Technology, Semiconductors, Electronics, Tech...","65,152",C. C. Wei,Morris Chang,https://www.tsmc.com/
2,85fa8c7dfbb447d898002cb3cfb2fffd,Nvidia Corporation,9,"26,914 Million USD","9,752 Million USD",For Year Ending JAN-2022,"44,187 Million USD","17,575 Million USD","26,612 Million USD",California,USA,Northern America,Americas,"[Technology, Semiconductors, Electronics, Tech...","18,975",Jensen Huang,"[Jensen Huang, Chris Malachowsky, Curtis Priem]",http://www.nvidia.com/
3,6b52bb9b50924235b1003c74bb8db227,Berkshire Hathaway,8,"354,640 Million USD","89,795 Million USD",For Year ending Dec-2021,"956,780 Million USD","443,850 Million USD","514,930 Million USD",Nebraska,USA,Northern America,Americas,"[Financial Services, Investments, Insurance]","372,000",Warren Buffett,Oliver Chace,https://www.berkshirehathaway.com/
4,d10c3bd4fe924f138454b6f0583de7a1,Meta Platforms,7,"117,930 Million USD","39,370 Million USD",Dec-2021,"165,990 Million USD","41,108 Million USD","124,880 Million USD",California,USA,Northern America,Americas,"[Communication Services, Internet or Mobile Ap...","77,805",Mark Zuckerberg,"[Mark Zuckerberg, Andrew McCollum, Eduardo Sav...",https://www.facebook.com/


In [162]:
to_drop, _ = columns_to_remove_not_remove(avengers_value)

Columns to remove:
id
world_rank
annual_net_income_in_usd
annual_results_for_year_ending
total_assets_in_usd
total_liabilities_in_usd
total_equity_in_usd
headquarters_region_city
headquarters_sub_region
headquarters_continent
founders


In [163]:
avengers_value = avengers_value.drop(to_drop, axis=1)
to_rename = rename_columns(avengers_value.columns)
avengers_value.columns = to_rename
avengers_value.head()

Columns to rename:
annual_revenue_in_usd -> revenue
headquarters_country -> country
company_business -> industry
number_of_employees -> employees
company_website -> links


,name,revenue,country,industry,employees,ceo,links
0,Apple,"394,330 Million USD",USA,"[Technology, Mobiles & Accessories, Electronic...","154,000",Tim Cook,https://www.apple.com/
1,Taiwan Semiconductor Manufacturing Company (tsmc),"55,306 Million USD",Taiwan,"[Technology, Semiconductors, Electronics, Tech...","65,152",C. C. Wei,https://www.tsmc.com/
2,Nvidia Corporation,"26,914 Million USD",USA,"[Technology, Semiconductors, Electronics, Tech...","18,975",Jensen Huang,http://www.nvidia.com/
3,Berkshire Hathaway,"354,640 Million USD",USA,"[Financial Services, Investments, Insurance]","372,000",Warren Buffett,https://www.berkshirehathaway.com/
4,Meta Platforms,"117,930 Million USD",USA,"[Communication Services, Internet or Mobile Ap...","77,805",Mark Zuckerberg,https://www.facebook.com/


In [ ]:
avengers_value.employees = avengers_value.employees.apply(lambda x: str(x).replace(',', ''))
avengers_disfold = avengers_disfold.replace('nan', np.nan)

avengers_value.employees = avengers_value.employees.astype(object).astype('Int64')

avengers_value.head()

In [164]:
cleaned_files.append(avengers_value)

## gren

### gren-companiesmarketcap.com.json

In [165]:
files_list[29]

'datasets/gren-companiesmarketcap.com.json'

In [166]:
gren_companies = pd.read_json(files_list[29])
gren_companies.head()

,name,market_capitalization_USD,price,country
0,Apple,2062000000000,130,USA
1,Saudi Aramco,1848000000000,8,S. Arabia
2,Microsoft,1676000000000,225,USA
3,Alphabet (Google),1137000000000,88,USA
4,Amazon,878160000000,86,USA


In [167]:
to_drop, _ = columns_to_remove_not_remove(gren_companies)

Columns to remove:
price


In [168]:
gren_companies = gren_companies.drop(to_drop, axis=1)
to_rename = rename_columns(gren_companies.columns)
gren_companies.columns = to_rename
gren_companies.head()

Columns to rename:
market_capitalization_USD -> marketcap


,name,marketcap,country
0,Apple,2062000000000,USA
1,Saudi Aramco,1848000000000,S. Arabia
2,Microsoft,1676000000000,USA
3,Alphabet (Google),1137000000000,USA
4,Amazon,878160000000,USA


In [169]:
cleaned_files.append(gren_companies)

### gren-disfold.com.json

In [170]:
files_list[30]

'datasets/gren-disfold.com.json'

In [338]:
gren_disfold = pd.read_json(files_list[30])
gren_disfold.head()

,name,market_capitalization_USD,stock,country,sector,industry
0,Apple Inc.,2825000000000,AAPL,United States,Technology,Consumer Electronics
1,Microsoft Corporation,2328000000000,MSFT,United States,Technology,Software—Infrastructure
2,Saudi Arabian Oil Company,2297000000000,2222,Saudi Arabia,Energy,Oil & Gas Integrated
3,Alphabet Inc.,1825000000000,GOOG,United States,Communication Services,Internet Content & Information
4,"Amazon.com, Inc.",1642000000000,AMZN,United States,Consumer Discretionary,Internet Retail


In [340]:
to_drop, _ = columns_to_remove_not_remove(gren_disfold)

Columns to remove:
stock


In [341]:
gren_disfold = gren_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(gren_disfold.columns)
gren_disfold.columns = to_rename
gren_disfold.head()

Columns to rename:
market_capitalization_USD -> marketcap
Column error: sector


,name,marketcap,country,sector,industry
0,Apple Inc.,2825000000000,United States,Technology,Consumer Electronics
1,Microsoft Corporation,2328000000000,United States,Technology,Software—Infrastructure
2,Saudi Arabian Oil Company,2297000000000,Saudi Arabia,Energy,Oil & Gas Integrated
3,Alphabet Inc.,1825000000000,United States,Communication Services,Internet Content & Information
4,"Amazon.com, Inc.",1642000000000,United States,Consumer Discretionary,Internet Retail


In [174]:
cleaned_files.append(gren_disfold)

### gren-ft.com.json

In [175]:
files_list[31]

'datasets/gren-ft.com.json'

In [343]:
gren_ft = pd.read_json(files_list[31])
gren_ft.head()

,name,country,sector,revenue_2020_EU,revenue_2017_EU,employees_number_2020,employees_number_2017,founding_year
0,Swappie,Finland,Technology,97611814,507000,218,1,2016
1,Kilo Health,Lithuania,Health,57318766,344428,177,10,2013
2,OCI,UK,Financial Services,568322073,4325512,32,4,2012
3,OnlyFans,UK,Technology,316732986,2652185,800,4,2016
4,Enpal,Germany,Energy,56109613,486165,365,9,2017


In [344]:
to_drop, _ = columns_to_remove_not_remove(gren_ft)

Columns to remove:
revenue_2017_EU
employees_number_2017


In [345]:
gren_ft = gren_ft.drop(to_drop, axis=1)
to_rename = rename_columns(gren_ft.columns)
gren_ft.columns = to_rename
gren_ft.head()

Columns to rename:
Column error: sector
revenue_2020_EU -> revenue
employees_number_2020 -> employees
founding_year -> founded


,name,country,sector,revenue,employees,founded
0,Swappie,Finland,Technology,97611814,218,2016
1,Kilo Health,Lithuania,Health,57318766,177,2013
2,OCI,UK,Financial Services,568322073,32,2012
3,OnlyFans,UK,Technology,316732986,800,2016
4,Enpal,Germany,Energy,56109613,365,2017


In [ ]:
gren_ft.employees = gren_ft.employees.astype(object).astype('Int64')

In [179]:
cleaned_files.append(gren_ft)

### gren-value.today.json

In [180]:
files_list[32]

'datasets/gren-value.today.json'

In [347]:
gren_value = pd.read_json(files_list[32])
gren_value.head()

,name,annual_revenue_USD,annual_net_income_USD,market_capitalization_2022,employees_number,CEO,headquarters_country,wikipedia_page_url,twitter_page_url,facebook_page_url
0,APPLE,394330000000,99803000000,2825000000000,154000,Tim Cook,USA,https://en.wikipedia.org/wiki/Apple_Inc.,https://twitter.com/apple?lang=en,https://www.facebook.com/apple/
1,MICROSOFT CORPORATION,198270000000,72738000000,2358000000000,220000,Satya Nadella,USA,https://en.wikipedia.org/wiki/Microsoft,https://twitter.com/Microsoft,https://www.facebook.com/Microsoft/
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),400460000000,105380000000,1889600000000,79800,Amin H. Al-Nasser,Saudi Arabia,https://en.wikipedia.org/wiki/Saudi_Aramco,https://twitter.com/Saudi_Aramco,https://www.facebook.com/aramco/
3,ALPHABET,257640000000,76003000000,1820000000000,156500,Sundar Pichai,USA,https://en.wikipedia.org/wiki/Alphabet_Inc.,https://twitter.com/google?lang=en,https://www.facebook.com/alphabet/
4,AMAZON.COM,469820000000,33364000000,1649000000000,1335000,Andy Jassy,USA,https://en.wikipedia.org/wiki/Amazon_(company),https://twitter.com/amazon,https://www.facebook.com/Amazon/


In [348]:
url_dataset = ['wikipedia_page_url', 'twitter_page_url', 'facebook_page_url']
gren_value['link'] = gren_value[url_dataset].values.tolist()

In [349]:
to_drop, _ = columns_to_remove_not_remove(gren_value)
to_drop.extend(url_dataset)

Columns to remove:
annual_net_income_USD


In [350]:
gren_value = gren_value.drop(to_drop, axis=1)
to_rename = rename_columns(gren_value.columns)
gren_value.columns = to_rename

Columns to rename:
annual_revenue_USD -> revenue
market_capitalization_2022 -> marketcap
employees_number -> employees
headquarters_country -> country
link -> links


,name,revenue,marketcap,employees,ceo,country,links
0,APPLE,394330000000,2825000000000,154000,Tim Cook,USA,"[https://en.wikipedia.org/wiki/Apple_Inc., htt..."
1,MICROSOFT CORPORATION,198270000000,2358000000000,220000,Satya Nadella,USA,"[https://en.wikipedia.org/wiki/Microsoft, http..."
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),400460000000,1889600000000,79800,Amin H. Al-Nasser,Saudi Arabia,"[https://en.wikipedia.org/wiki/Saudi_Aramco, h..."
3,ALPHABET,257640000000,1820000000000,156500,Sundar Pichai,USA,"[https://en.wikipedia.org/wiki/Alphabet_Inc., ..."
4,AMAZON.COM,469820000000,1649000000000,1335000,Andy Jassy,USA,[https://en.wikipedia.org/wiki/Amazon_(company...


In [351]:
gren_value.revenue = gren_value.revenue.astype(object).astype('Int64')
gren_value.marketcap = gren_value.marketcap.astype(object).astype('Int64')
gren_value.employees = gren_value.employees.astype(object).astype('Int64')
gren_value.head()

name         object
revenue      object
marketcap    object
employees    object
ceo          object
country      object
links        object
dtype: object

In [185]:
cleaned_files.append(gren_value)

## iGMM

### iGMM-cbinsights.com.json

In [186]:
files_list[33]

'datasets/iGMM-cbinsights.com.json'

In [352]:
igmm_cbinsights = pd.read_json(files_list[33])
igmm_cbinsights.head()

,company,valuation,datejoined,country,city,industry,selectinvestors
0,ByteDance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Canva,$40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [188]:
to_drop, _ = columns_to_remove_not_remove(igmm_cbinsights)

Columns to remove:
valuation
datejoined
city
selectinvestors


In [189]:
igmm_cbinsights = igmm_cbinsights.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_cbinsights.columns)
igmm_cbinsights.columns = to_rename
igmm_cbinsights.head()

Columns to rename:
company -> name


,name,country,industry
0,ByteDance,China,Artificial intelligence
1,SpaceX,United States,Other
2,SHEIN,China,E-commerce & direct-to-consumer
3,Stripe,United States,Fintech
4,Canva,Australia,Internet software & services


In [190]:
cleaned_files.append(igmm_cbinsights)

### iGMM-companiesmarketcap.com.json

In [191]:
files_list[34]

'datasets/iGMM-companiesmarketcap.com.json'

In [192]:
igmm_companies = pd.read_json(files_list[34])
igmm_companies.head()

,name,code,rank,marketcap,country,sharePrice,change(1day),change(1year),categories
0,Apple,AAPL,1,$2.242 T,USA,$140.94,-1.38%,-17.67%,"[Tech, Software, Dowjones, TechHardware, Elect..."
1,McDonald,MCD,46,$198.00 B,USA,$270.34,-0.53%,4.01%,"[Restaurantchains, Dowjones, Food]"
2,T-Mobile US,TMUS,51,$185.11 B,USA,$148.79,-0.40%,27.01%,[Telecommunication]
3,Toyota,TM,49,$195.66 B,Japan,$141.86,0.55%,-24.32%,"[Automakers, Manufacturing]"
4,Cisco,CSCO,47,$197.92 B,USA,$48.18,-0.84%,-17.05%,"[Tech, Electronics, Networkinghardware, Dowjon..."


In [193]:
to_drop, _ = columns_to_remove_not_remove(igmm_companies)

Columns to remove:
code
rank
sharePrice
change(1day)
change(1year)


In [194]:
igmm_companies = igmm_companies.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_companies.columns)
igmm_companies.columns = to_rename
igmm_companies.head()

Columns to rename:
categories -> industry


,name,marketcap,country,industry
0,Apple,$2.242 T,USA,"[Tech, Software, Dowjones, TechHardware, Elect..."
1,McDonald,$198.00 B,USA,"[Restaurantchains, Dowjones, Food]"
2,T-Mobile US,$185.11 B,USA,[Telecommunication]
3,Toyota,$195.66 B,Japan,"[Automakers, Manufacturing]"
4,Cisco,$197.92 B,USA,"[Tech, Electronics, Networkinghardware, Dowjon..."


In [195]:
cleaned_files.append(igmm_companies)

### iGMM-disfold.com.json

In [196]:
files_list[35]

'datasets/iGMM-disfold.com.json'

In [197]:
igmm_disfold = pd.read_json(files_list[35])
igmm_disfold.head()

,name,code,rank,ceo,country,sector,industry,headquarters,employees,founded,marketcap,revenue,net income,link
0,Apple,AAPL,1,Mr. Timothy D. Cook,United States,Technology,Consumer Electronics,"Cupertino, CA, United States","100,000","April 1, 1976",$2.825 Trillion,US$365.82 billion (2021),US$94.68 billion (2021),"[https://www.apple.com, https://www.facebook.c..."
1,ExxonMobil,XOM,23,Mr. Darren W. Woods,United States,Energy,Oil & Gas Integrated,"Irving, TX, United States","63,000","November 30, 1999",$348.72 Billion,US$276.692 billion (2021),US$23.040 billion (2021),"[https://corporate.exxonmobil.com, https://en...."
2,Roche,ROG,22,Dr. Severin Schwan Ph.D.,Switzerland,Healthcare,Drug Manufacturers—General,"Basel, Switzerland","100,920",None,US$349.50 Billion,None,None,[https://www.roche.com]
3,Home Depot,HD,25,Mr. Edward P. Decker,United States,Consumer Discretionary,Home Improvement Retail,"Atlanta, GA, United States","490,600","February 6, 1978",$321.47 Billion,US$110.2 billion (2019),US$11.24 billion (2019),"[https://www.homedepot.com, https://en.wikiped..."
4,Pfizer,PFE,27,"Dr. Albert Bourla D.V.M., DVM, Ph.D.",United States,Healthcare,Drug Manufacturers—General,"New York, NY, United States","79,000",1849,$300.79 Billion,$41.908 billion (2020),$9.616 billion (2020),"[https://www.pfizer.com, https://www.facebook...."


In [198]:
to_drop, _ = columns_to_remove_not_remove(igmm_disfold)
to_drop.append("headquarters")

Columns to remove:
code
rank
net income


In [199]:
igmm_disfold = igmm_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_disfold.columns)
igmm_disfold.columns = to_rename

Columns to rename:
Column error: sector
headquarters -> country
link -> links


,name,ceo,country,sector,industry,country,employees,founded,marketcap,revenue,links
0,Apple,Mr. Timothy D. Cook,United States,Technology,Consumer Electronics,"Cupertino, CA, United States","100,000","April 1, 1976",$2.825 Trillion,US$365.82 billion (2021),"[https://www.apple.com, https://www.facebook.c..."
1,ExxonMobil,Mr. Darren W. Woods,United States,Energy,Oil & Gas Integrated,"Irving, TX, United States","63,000","November 30, 1999",$348.72 Billion,US$276.692 billion (2021),"[https://corporate.exxonmobil.com, https://en...."
2,Roche,Dr. Severin Schwan Ph.D.,Switzerland,Healthcare,Drug Manufacturers—General,"Basel, Switzerland","100,920",None,US$349.50 Billion,None,[https://www.roche.com]
3,Home Depot,Mr. Edward P. Decker,United States,Consumer Discretionary,Home Improvement Retail,"Atlanta, GA, United States","490,600","February 6, 1978",$321.47 Billion,US$110.2 billion (2019),"[https://www.homedepot.com, https://en.wikiped..."
4,Pfizer,"Dr. Albert Bourla D.V.M., DVM, Ph.D.",United States,Healthcare,Drug Manufacturers—General,"New York, NY, United States","79,000",1849,$300.79 Billion,$41.908 billion (2020),"[https://www.pfizer.com, https://www.facebook...."


In [ ]:
igmm_disfold.employees = igmm_disfold.employees.apply(lambda x: str(x).replace(',', ''))
igmm_disfold.founded = igmm_disfold.founded.str.extract(r'([\d]{4})')

igmm_disfold.employees = igmm_disfold.employees.astype(object).astype('Int64')
igmm_disfold.founded = igmm_disfold.founded.astype(object).astype('Int64')

igmm_disfold.head()

In [200]:
cleaned_files.append(igmm_disfold)

### iGMM-value.today.json

In [201]:
files_list[36]

'datasets/iGMM-value.today.json'

In [202]:
igmm_value = pd.read_json(files_list[36])
igmm_value.head()

,name,ceo,market value (Jan-07-2022),world rank (Jan-07-2022),market value (Jan 1st 2020),world rank (Jan-2020),headquarters country,number of employees,company business,annual revenue in USD,annual net income in USD,company website
0,APPLE,Tim Cook,"2,825.000 Billion USD",1,"1,323.000 Billion USD",2.0,USA,"154,000","[Technology, Mobiles & Accessories, Electronics]","365,820 Million USD","94,680 Million USD",https://www.apple.com/
1,MICROSOFT CORPORATION,Satya Nadella,"2,358.000 Billion USD",2,"1,215.000 Billion USD",3.0,USA,"220,000","[Technology, Software and IT, Laptops]","198,270 Million USD","72,738 Million USD",https://www.microsoft.com/
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),Amin H. Al-Nasser,"1,889.600 Billion USD",3,"1,898.100 Billion USD",1.0,Saudi Arabia,"79,800","[Energy, Oil and Gas, Chemicals]","400,460 Million USD","105,380 Million USD",https://www.saudiaramco.com/
3,ALPHABET,Sundar Pichai,"1,820.000 Billion USD",4,943.897 Billion USD,4.0,USA,"156,500","[Technology, Internet or Mobile App Based Busi...","257,640 Million USD","76,003 Million USD",https://abc.xyz/
4,AMAZON.COM,Andy Jassy,"1,649.000 Billion USD",5,941.028 Billion USD,5.0,USA,"1,335,000","[eCommerce, Internet or Mobile App Based Busin...","469,820 Million USD","33,364 Million USD",https://www.amazon.com/


In [203]:
to_drop, _ = columns_to_remove_not_remove(igmm_value)

Columns to remove:
market value (Jan-07-2022)
world rank (Jan-07-2022)
market value (Jan 1st 2020)
world rank (Jan-2020)
annual net income in USD
company website


In [204]:
igmm_value = igmm_value.drop(to_drop, axis=1)
to_rename = rename_columns(igmm_value.columns)
igmm_value.columns = to_rename

Columns to rename:
headquarters country -> country
number of employees -> employees
company business -> industry
annual revenue in USD -> revenue


,name,ceo,country,employees,industry,revenue
0,APPLE,Tim Cook,USA,"154,000","[Technology, Mobiles & Accessories, Electronics]","365,820 Million USD"
1,MICROSOFT CORPORATION,Satya Nadella,USA,"220,000","[Technology, Software and IT, Laptops]","198,270 Million USD"
2,SAUDI ARABIAN OIL COMPANY (Saudi Aramco),Amin H. Al-Nasser,Saudi Arabia,"79,800","[Energy, Oil and Gas, Chemicals]","400,460 Million USD"
3,ALPHABET,Sundar Pichai,USA,"156,500","[Technology, Internet or Mobile App Based Busi...","257,640 Million USD"
4,AMAZON.COM,Andy Jassy,USA,"1,335,000","[eCommerce, Internet or Mobile App Based Busin...","469,820 Million USD"


In [ ]:
igmm_value.employees = igmm_disfold.employees.apply(lambda x: str(x).replace(',', ''))

igmm_value.employees = igmm_value.employees.astype(object).astype('Int64')

igmm_value.head()

In [205]:
cleaned_files.append(igmm_value)

## silvestri

### silvestri-disfold.com.csv

In [206]:
files_list[37]

'datasets/silvestri-disfold.com.csv'

In [207]:
silvestri_disfold = pd.read_csv(files_list[37])
silvestri_disfold.head()

,name,country,market_value_apr_2022,sector,stock
0,Apple Inc.,united states,$2.825 T,technology,AAPL
1,Microsoft Corporation,united states,$2.328 T,technology,MSFT
2,Saudi Arabian Oil Company,saudi arabia,$2.297 T,energy,2222
3,Alphabet Inc.,united states,$1.825 T,communication services,GOOG
4,"Amazon.com, Inc.",united states,$1.642 T,consumer discretionary,AMZN


In [208]:
to_drop, _ = columns_to_remove_not_remove(silvestri_disfold)

Columns to remove:
market_value_apr_2022
stock


In [209]:
silvestri_disfold = silvestri_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_disfold.columns)
silvestri_disfold.columns = to_rename
silvestri_disfold.head()

Columns to rename:
Column error: sector


,name,country,sector
0,Apple Inc.,united states,technology
1,Microsoft Corporation,united states,technology
2,Saudi Arabian Oil Company,saudi arabia,energy
3,Alphabet Inc.,united states,communication services
4,"Amazon.com, Inc.",united states,consumer discretionary


In [210]:
cleaned_files.append(silvestri_disfold)

### silvestri-forbes.com.csv

In [211]:
files_list[38]

'datasets/silvestri-forbes.com.csv'

In [212]:
silvestri_forbes = pd.read_csv(files_list[38])
silvestri_forbes.head()

,country,market_value_apr_2022,name,revenue_2022
0,united states,$741.48 B,berkshire hathaway,$276.09 B
1,china,$214.43 B,icbc,$208.13 B
2,saudi arabia,"$2,292.08 B",saudi arabian oil company (saudi aramco),$400.38 B
3,united states,$374.45 B,jpmorgan chase,$124.54 B
4,china,$181.32 B,china construction bank,$202.07 B


In [213]:
to_drop, _ = columns_to_remove_not_remove(silvestri_forbes)

Columns to remove:
market_value_apr_2022


In [214]:
silvestri_forbes = silvestri_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_forbes.columns)
silvestri_forbes.columns = to_rename
silvestri_forbes.head()

Columns to rename:
revenue_2022 -> revenue


,country,name,revenue
0,united states,berkshire hathaway,$276.09 B
1,china,icbc,$208.13 B
2,saudi arabia,saudi arabian oil company (saudi aramco),$400.38 B
3,united states,jpmorgan chase,$124.54 B
4,china,china construction bank,$202.07 B


In [215]:
cleaned_files.append(silvestri_forbes)

### silvestri-ft.com.csv

In [216]:
files_list[39]

'datasets/silvestri-ft.com.csv'

In [217]:
silvestri_ft = pd.read_csv(files_list[39])
silvestri_ft.head()

,country,employees_2017,employees_2020,founding_year,name,revenue_2017,revenue_2020,sector
0,finland,1,218.0,2016,swappie,"507,000","97,611,814",technology
1,lithuania,10,177.0,2013,kilo health,"344,428","57,318,766",health
2,uk,4,32.0,2012,oci,"4,325,512","568,322,073",financial services
3,uk,4,800.0,2016,onlyfans,"2,652,185","316,732,986",technology
4,germany,9,365.0,2017,enpal,"486,165","56,109,613",energy


In [218]:
to_drop, _ = columns_to_remove_not_remove(silvestri_ft)

Columns to remove:
employees_2017
revenue_2017


In [219]:
silvestri_ft = silvestri_ft.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_ft.columns)
silvestri_ft.columns = to_rename

Columns to rename:
employees_2020 -> employees
founding_year -> founded
revenue_2020 -> revenue
Column error: sector


,country,employees,founded,name,revenue,sector
0,finland,218.0,2016,swappie,"97,611,814",technology
1,lithuania,177.0,2013,kilo health,"57,318,766",health
2,uk,32.0,2012,oci,"568,322,073",financial services
3,uk,800.0,2016,onlyfans,"316,732,986",technology
4,germany,365.0,2017,enpal,"56,109,613",energy


In [ ]:
silvestri_ft.revenue = silvestri_ft.revenue.apply(lambda x : (str(x).replace(',', '')))
silvestri_ft.employees = silvestri_ft.employees.astype(str).replace(r'\.0$', '', regex=True)
silvestri_ft = silvestri_ft.replace('nan', np.nan)

silvestri_ft.revenue = silvestri_ft.revenue.astype(object).astype('Int64')
silvestri_ft.employees = silvestri_ft.employees.astype(object).astype('Int64')
silvestri_ft.founded = silvestri_ft.founded.astype(object).astype('Int64')

silvestri_ft.head()

In [220]:
cleaned_files.append(silvestri_ft)

### silvestri-value.today.csv

In [221]:
files_list[40]

'datasets/silvestri-value.today.csv'

In [222]:
silvestri_value = pd.read_csv(files_list[40])
silvestri_value.head()

,ceo,country,market_value_jan_2020,market_value_jan_2021,market_value_jan_2022,name,employees_2022,revenue_2022,sector
0,Bernard Arnault,france,237.807 Billion USD,313.500 Billion USD,409.480 Billion USD,lvmh,"150,000","70,765 Million USD",fashion and luxury goods
1,Ulf Mark Schneider,switzerland,311.592 Billion USD,336.110 Billion USD,372.720 Billion USD,nestle ag,"276,000","92,575 Million USD",consumer defensive
2,Severin Schwan,switzerland,279.684 Billion USD,295.780 Billion USD,356.150 Billion USD,roche holding ag,"100,920","66,757 Million USD",healthcare
3,Peter Wennink,netherlands,127.542 Billion USD,205.120 Billion USD,307.330 Billion USD,asml holding,"28,000","20,509 Million USD",technology
4,Bob van Dijk,netherlands,125.912 Billion USD,176.280 Billion USD,254.550 Billion USD,prosus,"23,874","6,866 Million USD",communication services


In [223]:
to_drop, _ = columns_to_remove_not_remove(silvestri_value)

Columns to remove:
market_value_jan_2020
market_value_jan_2021
market_value_jan_2022


In [224]:
silvestri_value = silvestri_value.drop(to_drop, axis=1)
to_rename = rename_columns(silvestri_value.columns)
silvestri_value.columns = to_rename

Columns to rename:
employees_2022 -> employees
revenue_2022 -> revenue
Column error: sector


,ceo,country,name,employees,revenue,sector
0,Bernard Arnault,france,lvmh,"150,000","70,765 Million USD",fashion and luxury goods
1,Ulf Mark Schneider,switzerland,nestle ag,"276,000","92,575 Million USD",consumer defensive
2,Severin Schwan,switzerland,roche holding ag,"100,920","66,757 Million USD",healthcare
3,Peter Wennink,netherlands,asml holding,"28,000","20,509 Million USD",technology
4,Bob van Dijk,netherlands,prosus,"23,874","6,866 Million USD",communication services


In [ ]:
silvestri_value.employees = silvestri_value.employees.apply(lambda x : (str(x).replace(',', '')))
silvestri_value = silvestri_value.replace('nan', np.nan)

silvestri_value.employees = silvestri_value.employees.astype(object).astype('Int64')

silvestri_value.head()

In [225]:
cleaned_files.append(silvestri_value)

## slytherin

### slytherin-disfold.com.json

In [226]:
files_list[41]

'datasets/slytherin-disfold.com.json'

In [353]:
slytherin_disfold = pd.read_json(files_list[41])
slytherin_disfold.head()

,link,name,headquarters,employees,ceo,market_cap
0,https://disfold.com/company/great-wall-motor-c...,Great Wall Motor Co Ltd,"Baoding, China","63,174",Not found,CN¥203.69 Billion
1,https://disfold.com/company/colbun-sa/,Colbún SA,"Santiago, Chile","1,193",Mr. Thomas Lippold Keller,CL$1.098 Trillion
2,https://disfold.com/company/usinas-siderurgica...,Usinas Siderúrgicas de Minas Gerais SA,"Belo Horizonte, Brazil",Not found,Mr. Sergio Leite de Andrade,€3.54 Billion
3,https://disfold.com/company/origin-enterprises...,Origin Enterprises plc,"Dublin, Ireland","2,641",Mr. Sean Gerard Coyle,€539.3 Million
4,https://disfold.com/company/summit-real-estate...,Summit Real Estate Holdings Ltd,"Haifa, Israel",144,Mr. Amir Sagi,₪45.1 Million


#### Marketcap diverse valute

In [354]:
to_drop, _ = columns_to_remove_not_remove(slytherin_disfold)
to_drop.append("market_cap")

Columns to remove:


In [355]:
slytherin_disfold = slytherin_disfold.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_disfold.columns)
slytherin_disfold.columns = to_rename

Columns to rename:
link -> links
headquarters -> country


In [357]:
slytherin_disfold.country = slytherin_disfold.country.apply(lambda x : (str(x).split(',')[-1].strip()))
slytherin_disfold.employees = slytherin_disfold.employees.apply(lambda x : (str(x).replace(',', '')))
slytherin_disfold = slytherin_disfold.replace('Not found', np.nan)

slytherin_disfold.employees = slytherin_disfold.employees.astype(object).astype('Int64')

slytherin_disfold.head()

,links,name,country,employees,ceo
0,https://disfold.com/company/great-wall-motor-c...,Great Wall Motor Co Ltd,China,63174,NaN
1,https://disfold.com/company/colbun-sa/,Colbún SA,Chile,1193,Mr. Thomas Lippold Keller
2,https://disfold.com/company/usinas-siderurgica...,Usinas Siderúrgicas de Minas Gerais SA,Brazil,<NA>,Mr. Sergio Leite de Andrade
3,https://disfold.com/company/origin-enterprises...,Origin Enterprises plc,Ireland,2641,Mr. Sean Gerard Coyle
4,https://disfold.com/company/summit-real-estate...,Summit Real Estate Holdings Ltd,Israel,144,Mr. Amir Sagi


In [230]:
cleaned_files.append(slytherin_disfold)

### slytherin-forbes.com.json

In [231]:
files_list[42]

'datasets/slytherin-forbes.com.json'

In [358]:
slytherin_forbes = pd.read_json(files_list[42])
slytherin_forbes.head()

,link,name,industry,founded,country,ceo,employees,revenue
0,https://www.forbes.com/companies/berkshire-hat...,Berkshire Hathaway,Diversified Financials,1939,United States,Warren Edward Buffett,"372,000",$276.1B
1,https://www.forbes.com/companies/icbc,ICBC,Banking,1984,China,Shu Gu,"434,089",$208.1B
2,https://www.forbes.com/companies/saudi-arabian...,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,1933,Saudi Arabia,Amin bin Hasan Al-Nasser,"68,493",$400.4B
3,https://www.forbes.com/companies/jpmorgan-chase,JPMorgan Chase,Banking and Financial Services,2000,United States,Jamie Dimon,"270,000",$124.5B
4,https://www.forbes.com/companies/china-constru...,China Construction Bank,Banking,2014,China,Wang Zuji,"349,671",$202.1B


In [359]:
to_drop, _ = columns_to_remove_not_remove(slytherin_forbes)
to_drop.append("link")

Columns to remove:


In [360]:
slytherin_forbes = slytherin_forbes.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_forbes.columns)
slytherin_forbes.columns = to_rename

Columns to rename:


In [363]:
slytherin_forbes.employees = slytherin_forbes.employees.apply(lambda x : (str(x).replace(',', '')))
slytherin_forbes = slytherin_forbes.replace('Not found', np.nan)

slytherin_forbes.employees = slytherin_forbes.employees.astype(object).astype('Int64')

slytherin_forbes.head()

,name,industry,founded,country,ceo,employees,revenue
0,Berkshire Hathaway,Diversified Financials,1939,United States,Warren Edward Buffett,372000,$276.1B
1,ICBC,Banking,1984,China,Shu Gu,434089,$208.1B
2,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,1933,Saudi Arabia,Amin bin Hasan Al-Nasser,68493,$400.4B
3,JPMorgan Chase,Banking and Financial Services,2000,United States,Jamie Dimon,270000,$124.5B
4,China Construction Bank,Banking,2014,China,Wang Zuji,349671,$202.1B


In [235]:
cleaned_files.append(slytherin_forbes)

### slytherin-ft.com.json

In [236]:
files_list[43]

'datasets/slytherin-ft.com.json'

In [365]:
slytherin_ft = pd.read_json(files_list[43])
slytherin_ft.head()

,link,name,country,industry,revenue,employees,founded
0,https://www.greatdealscorp.com/,Great Deals E-Commerce Corp.,Philippines,Ecommerce,88.77,385,2017
1,http://www.pclchip.com/,PCL Inc.,South Korea,Pharmaceuticals,45.48,63,2008
2,https://www.spolive.com/,Spolive,South Korea,Games industry,9.49,47,2013
3,https://www.workmate.asia/,Workmate,Singapore,Support Services,12.44,82,2015
4,http://www.taimedbiologics.com/,TaiMed Biologics,Taiwan,Pharmaceuticals,24.42,78,2007


In [366]:
to_drop, _ = columns_to_remove_not_remove(slytherin_ft)

Columns to remove:


In [367]:
slytherin_ft = slytherin_ft.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_ft.columns)
slytherin_ft.columns = to_rename

Columns to rename:
link -> links


,links,name,country,industry,revenue,employees,founded
0,https://www.greatdealscorp.com/,Great Deals E-Commerce Corp.,Philippines,Ecommerce,88.77,385,2017
1,http://www.pclchip.com/,PCL Inc.,South Korea,Pharmaceuticals,45.48,63,2008
2,https://www.spolive.com/,Spolive,South Korea,Games industry,9.49,47,2013
3,https://www.workmate.asia/,Workmate,Singapore,Support Services,12.44,82,2015
4,http://www.taimedbiologics.com/,TaiMed Biologics,Taiwan,Pharmaceuticals,24.42,78,2007


In [371]:
slytherin_ft.isnull().sum()

links        46
name          0
country       0
industry      0
revenue       0
employees     0
founded       0
dtype: int64

In [373]:
slytherin_ft.employees = slytherin_ft.employees.apply(lambda x : (str(x).replace('.', '')))
slytherin_ft.revenue = slytherin_ft.revenue.apply(lambda x : (str(x).replace(',', '')))
slytherin_ft = slytherin_ft.replace('Not found', np.nan)
slytherin_ft = slytherin_ft.replace('nan', np.nan)

slytherin_ft.revenue = slytherin_ft.revenue.astype(object).astype('Int64')
slytherin_ft.employees = slytherin_ft.employees.astype(object).astype('Int64')
slytherin_ft.founded = slytherin_ft.founded.astype(object).astype('Int64')

slytherin_ft.head()

,links,name,country,industry,revenue,employees,founded
0,https://www.greatdealscorp.com/,Great Deals E-Commerce Corp.,Philippines,Ecommerce,8877,385,2017
1,http://www.pclchip.com/,PCL Inc.,South Korea,Pharmaceuticals,4548,63,2008
2,https://www.spolive.com/,Spolive,South Korea,Games industry,949,47,2013
3,https://www.workmate.asia/,Workmate,Singapore,Support Services,1244,82,2015
4,http://www.taimedbiologics.com/,TaiMed Biologics,Taiwan,Pharmaceuticals,2442,78,2007


In [240]:
cleaned_files.append(slytherin_ft)

### slytherin-value.today.json

In [241]:
files_list[44]

'datasets/slytherin-value.today.json'

In [381]:
slytherin_value = pd.read_json(files_list[44])
slytherin_value.head()

,link,rank,name,ceo,founded,revenue,country,industry
0,https://www.value.today/company/apple,1,APPLE,Tim Cook,1976,"365,820 Million USD",USA,"[Technology, Mobiles & Accessories, Electronic..."
1,https://www.value.today/company/taiwan-semicon...,10,TAIWAN SEMICONDUCTOR MANUFACTURING COMPANY (TSMC),Mark Liu,1987,"55,306 Million USD",Taiwan,"[Technology, Semiconductors, Electronics, Tech..."
2,https://www.value.today/company/microsoft-corp...,2,MICROSOFT CORPORATION,Satya Nadella,1975,"198,270 Million USD",USA,"[Technology, Software and IT, Laptops, Video G..."
3,https://www.value.today/company/nvidia-corpora...,9,NVIDIA CORPORATION,Jensen Huang,1993,"26,914 Million USD",USA,"[Technology, Semiconductors, Electronics, Tech..."
4,https://www.value.today/company/amazon.com,5,AMAZON.COM,Andy Jassy,1994,"469,820 Million USD",USA,"[eCommerce, Internet or Mobile App Based Busin..."


In [382]:
to_drop, _ = columns_to_remove_not_remove(slytherin_value)
to_drop.append("link")

Columns to remove:
rank


In [383]:
slytherin_value = slytherin_value.drop(to_drop, axis=1)
to_rename = rename_columns(slytherin_value.columns)
slytherin_value.columns = to_rename

Columns to rename:


In [385]:
slytherin_value.founded = slytherin_value.founded.str.extract(r'([\d]{4})')
slytherin_value = slytherin_value.replace('Not found', np.nan)
slytherin_value = slytherin_value.replace('nan', np.nan)

slytherin_value.founded = slytherin_value.founded.astype(object).astype('Int64')
slytherin_value.head()

,name,ceo,founded,revenue,country,industry
0,APPLE,Tim Cook,1976,"365,820 Million USD",USA,"[Technology, Mobiles & Accessories, Electronic..."
1,TAIWAN SEMICONDUCTOR MANUFACTURING COMPANY (TSMC),Mark Liu,1987,"55,306 Million USD",Taiwan,"[Technology, Semiconductors, Electronics, Tech..."
2,MICROSOFT CORPORATION,Satya Nadella,1975,"198,270 Million USD",USA,"[Technology, Software and IT, Laptops, Video G..."
3,NVIDIA CORPORATION,Jensen Huang,1993,"26,914 Million USD",USA,"[Technology, Semiconductors, Electronics, Tech..."
4,AMAZON.COM,Andy Jassy,1994,"469,820 Million USD",USA,"[eCommerce, Internet or Mobile App Based Busin..."


In [245]:
cleaned_files.append(slytherin_value)

## wissel

### wissel-ariregister.csv

In [246]:
files_list[45]

'datasets/wissel-ariregister.rik.ee.csv'

In [247]:
wissel_ariregister = pd.read_csv(files_list[45])
wissel_ariregister.head()

,Unnamed: 0,ID azienda,Name,Capital,Address,Area of Activity
0,0,1,10DspaceOÃœ(14318875),NaN,NaN,NaN
1,1,2,21SparrowsOÃœ(16270291),Capitalis10Â 000Â â‚¬,Harju maakond Tallinn ...,Other activities auxiliary to financial servic...
2,2,3,3DSTOÃœ(12454634),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...,Specialised design activities
3,3,4,8BitsSpaceOÃœ(14867109),Capitalis2Â 500Â â‚¬,Harju maakond Tallinn ...,Computer programming activities
4,4,5,A1EhituspartnerOÃœ(16451101),Capitalis2Â 500Â â‚¬,PÃµlva maakond PÃµlva ...,Construction of residential and non-residentia...


In [248]:
to_drop, _ = columns_to_remove_not_remove(wissel_ariregister)

Columns to remove:
Unnamed: 0
ID azienda
Capital


In [249]:
wissel_ariregister = wissel_ariregister.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_ariregister.columns)
wissel_ariregister.columns = to_rename
wissel_ariregister.head()

Columns to rename:
Address -> country
Area of Activity -> industry


,name,country,industry
0,10DspaceOÃœ(14318875),NaN,NaN
1,21SparrowsOÃœ(16270291),Harju maakond Tallinn ...,Other activities auxiliary to financial servic...
2,3DSTOÃœ(12454634),Harju maakond Tallinn ...,Specialised design activities
3,8BitsSpaceOÃœ(14867109),Harju maakond Tallinn ...,Computer programming activities
4,A1EhituspartnerOÃœ(16451101),PÃµlva maakond PÃµlva ...,Construction of residential and non-residentia...


In [250]:
cleaned_files.append(wissel_ariregister)

### wissel-companiesmarketcap.com.csv

In [251]:
files_list[46]

'datasets/wissel-companiesmarketcap.com.csv'

In [386]:
wissel_companies = pd.read_csv(files_list[46])
wissel_companies.head()

,Unnamed: 0,ID,Name,Company code,Marketcap,Share price,Earnings,Revenue,Shares,Employees
0,0,1.0,Vertex Pharmaceuticals,VRTX,$77.24 B,$300.94,$4.08 B,$8.70 B,256.500.000,3.9
1,1,2.0,Verano Holdings,VRNOF,$0.97 B,$3.00,-$62.2 M,$0.81 B,NaN,3.888
2,2,3.0,AZZ,AZZ,$1.09 B,$43.85,$0.12 B,$1.17 B,NaN,3.885
3,3,4.0,F.N.B. Corporation,FNB,$4.83 B,$13.41,$0.50 B,$1.33 B,NaN,3.884
4,4,5.0,Quanex Building Products,NX,$0.82 B,$24.96,$0.10 B,$1.22 B,NaN,3.86


In [387]:
to_drop, _ = columns_to_remove_not_remove(wissel_companies)

Columns to remove:
Unnamed: 0
ID
Company code
Share price
Earnings
Shares


In [388]:
wissel_companies = wissel_companies.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_companies.columns)
wissel_companies.columns = to_rename

Columns to rename:


In [391]:
wissel_companies.employees = wissel_companies.employees.apply(lambda x : (str(x).replace('.', '')))
wissel_companies = wissel_companies.replace('nan', np.nan)
wissel_companies.employees = wissel_companies.employees.astype(object).astype('Int64')

wissel_companies.head()

,name,marketcap,revenue,employees
0,Vertex Pharmaceuticals,$77.24 B,$8.70 B,39
1,Verano Holdings,$0.97 B,$0.81 B,3888
2,AZZ,$1.09 B,$1.17 B,3885
3,F.N.B. Corporation,$4.83 B,$1.33 B,3884
4,Quanex Building Products,$0.82 B,$1.22 B,386


In [255]:
cleaned_files.append(wissel_companies)

### wissel-gov.uk.csv

In [256]:
files_list[47]

'datasets/wissel-gov.uk.csv'

In [257]:
wissel_gov = pd.read_csv(files_list[47])
wissel_gov.head()

,URL,ID,Name,Company ID,Company Status,Company Type,Registration Date,Incorporation Date,Dissolution Date,Office Address
0,https://find-and-update.company-information.se...,1,A LIMITED,11790215,Active,PrivatelimitedCompany,NaN,28 January 2019,NaN,38SpringfieldRoad Gillingham Kent England ME71YJ
1,https://find-and-update.company-information.se...,2,A LIMITED,05438136,Dissolved,PrivatelimitedCompany,NaN,27 April 2005,4 December 2012,60WindsorAvenue London SW192RR
2,https://find-and-update.company-information.se...,3,A & P (ESTATES & INVESTMENTS) LIMITED,OE000110,Registered,Overseasentity,19 August 2022,NaN,NaN,67WestowStreet UpperNorwood London England SE1...
3,https://find-and-update.company-information.se...,4,A AND U INVESTMENTS LIMITED,OE004228,Registered,Overseasentity,17 November 2022,NaN,NaN,OrdnanceHouse 31PierRoad St.Helier Jersey JE48PW
4,https://find-and-update.company-information.se...,5,ACRIS COURT 27 A LIMITED,OE004120,Registered,Overseasentity,16 November 2022,NaN,NaN,SchindlersTrustMauritiusLimited2ndFloor BlockB...


In [258]:
to_drop, _ = columns_to_remove_not_remove(wissel_gov)

Columns to remove:
URL
ID
Company ID
Company Status
Company Type
Registration Date
Incorporation Date
Dissolution Date


In [259]:
wissel_gov = wissel_gov.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_gov.columns)
wissel_gov.columns = to_rename
wissel_gov.head()

Columns to rename:
Office Address -> country


,name,country
0,A LIMITED,38SpringfieldRoad Gillingham Kent England ME71YJ
1,A LIMITED,60WindsorAvenue London SW192RR
2,A & P (ESTATES & INVESTMENTS) LIMITED,67WestowStreet UpperNorwood London England SE1...
3,A AND U INVESTMENTS LIMITED,OrdnanceHouse 31PierRoad St.Helier Jersey JE48PW
4,ACRIS COURT 27 A LIMITED,SchindlersTrustMauritiusLimited2ndFloor BlockB...


In [260]:
cleaned_files.append(wissel_gov)

### wissel-info-clipper.com.csv

In [1]:
files_list[48]

NameError: name 'files_list' is not defined

In [262]:
wissel_info = pd.read_csv(files_list[48])
wissel_info.head()

,Unnamed: 0,INDEX,URL,Name,Trade Name,Address Name,Postalcode,City,State,Country,Location type
0,0,0,https://www.info-clipper.com/en/company/united...,"CNHI, LLC",NaN,445 DEXTER AVE,361043892,MONTGOMERY,Alabama(AL),UNITED STATES,Headquarter
1,1,1,https://www.info-clipper.com/en/company/united...,CRAIG & ROSE LIMITED,NaN,7th Floor Suite 1 50 Broadway,SW1H 0BL,LONDON,UNITED KINGDOM,UNITED KINGDOM,Single address
2,2,2,https://www.info-clipper.com/en/company/italy/...,SEGRETARIATO GENERALE DELLA PRESIDENZA DELLA R...,NaN,VIA DELLA DATARIA 96,187,ROMA,ITALY,ITALY,Headquarter
3,3,3,https://www.info-clipper.com/en/company/estoni...,BIH REAL ESTATE OU,NaN,Narva mnt 6-44,10117,Tallinn,ESTONIA,ESTONIA,Single address
4,4,4,https://www.info-clipper.com/en/company/united...,"DEVELOPMENTAL SERVICES, CALIFORNIA DEPARTMENT OF",NaN,1215 O ST,958145804,SACRAMENTO,California(CA),UNITED STATES,Headquarter


In [263]:
to_drop, _ = columns_to_remove_not_remove(wissel_info)

Columns to remove:
Unnamed: 0
INDEX
URL
Trade Name
Address Name
Postalcode
City
State
Location type


In [264]:
wissel_info = wissel_info.drop(to_drop, axis=1)
to_rename = rename_columns(wissel_info.columns)
wissel_info.columns = to_rename
wissel_info.head()

Columns to rename:


,name,country
0,"CNHI, LLC",UNITED STATES
1,CRAIG & ROSE LIMITED,UNITED KINGDOM
2,SEGRETARIATO GENERALE DELLA PRESIDENZA DELLA R...,ITALY
3,BIH REAL ESTATE OU,ESTONIA
4,"DEVELOPMENTAL SERVICES, CALIFORNIA DEPARTMENT OF",UNITED STATES


In [265]:
cleaned_files.append(wissel_info)

# Phase 2

In [266]:
cleaned_col={}
for file in cleaned_files:
    
    for f in file.columns:
        if f not in cleaned_col:
            cleaned_col[f] = 0
        cleaned_col[f] += 1


In [267]:
len(cleaned_files)

47

In [268]:
cleaned_col

{'name': 47,
 'country': 45,
 'industry': 24,
 'founded': 20,
 'marketcap': 17,
 'sector': 14,
 'revenue': 18,
 'employees': 19,
 'links': 10,
 'ceo': 12}

### Sistemare campi marketcap (valute e unità di misura diverse) e employees.
### sector/industry?